# Imports and file details

In [1]:
import pyarrow as pa
import pyarrow.parquet as pq
import pyarrow.compute as pc

import matplotlib.pyplot as plt
import seaborn as sns

sns.set_theme()
sns.set_style("ticks")

import os
import sys
sys.path.insert(0,'..')

import pandas as pd
import json
import numpy as np

import dask.dataframe as dd
from dask.dataframe.utils import make_meta

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', -1)

from src_new.utils import SQLParserSchema, PGLastSchema, get_file_encodings, SQLGlotSchema, SimpleDDLParserSchema

from dask.diagnostics import ProgressBar
ProgressBar().register()

C:\Users\geacu\AppData\Local\Temp\ipykernel_14044\2489133733.py:27: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('max_colwidth', -1)


In [2]:
def read_partial(parser, schema_type, cols):
    _outdir = '../out_new/'

    if parser == 'sqlparser':
        p = SQLParserSchema()
        _outdir += 'sqlparser'
    elif parser == 'pglast':
        p = PGLastSchema()
        _outdir += 'pglast'
    elif parser == 'sqlglot':
        p = SQLGlotSchema()
        _outdir += 'sqlglot'
    elif parser == 'simple':
        p = SimpleDDLParserSchema()
        _outdir += 'simpleddlparser'
    
    if schema_type == 'file':
        _schema = p.file_level_schema
        _outdir += '/'
    elif schema_type == 'stmt':
        _schema = p.statement_list_sch
        _outdir += '_details/'
    
    ddf = dd.read_parquet(_outdir,columns=cols, schema=_schema, split_row_groups=True, calculate_divisions=True,engine='pyarrow')
    return ddf

In [3]:
# Read file details
fd = []
for file in os.listdir('../out_new/filedetails/'):
        full_filename = "%s/%s" % ('../out_new/filedetails/', file)
        with open(full_filename,'r') as fi:
            dict = json.load(fi)
            for item in dict:
                  fd.append(item)

filedetails_df = pd.DataFrame.from_dict(fd)
del fd

# SQLGLOT

In [4]:
sqlglot_ddf = read_partial('sqlglot','file',SQLGlotSchema().file_level_schema.names)
sqlparser_details_ddf = read_partial('sqlglot','stmt',SQLGlotSchema().statement_list_sch.names)

In [5]:
sqlparser_details_ddf.head()

[########################################] | 100% Completed | 791.03 ms


,file_id,statement_id,parsed,parsed_none,parsed_postgres,parsed_mysql,parsed_tsql,dialect,parse_error,file_parse_error_start_context,file_parse_error_highlight,num_statements,num_distinct_tables,table_list,num_distinct_columns,column_list,num_distinct_schemas,schema_list,num_distinct_dbs,db_list,view_list,num_constraints,num_ctr_notnull,num_ctr_unique,num_ctr_primary,num_ctr_foreign,comment_count,num_drop_stmt,num_drop_like_stmt,num_create_stmt,num_create_like_stmt,num_insert_stmt,num_insert_like_stmt,num_alter_stmt,num_alter_like_stmt,num_select_stmt,num_view_stmt,num_truncate_stmt,num_update_stmt,num_comment_stmt,num_delete_stmt,num_set_like_stmt,num_execute_like_stmt,num_index_like_stmt,num_transaction_like_stmt,num_command_stmt,num_command_alter_stmt,counter_str,original
0,419753,0,0,NaN,NaN,NaN,NaN,None,Expecting )|Invalid expression / Unexpected token|,"more info\n\n CREATE TABLE ""SPACEUSER"".""RHNSNAPSHOTCONFIGCHANNEL""\n (\t""SNAPSHOT_ID"" NUMBER NOT NULL |",ENABLE|,0,0,None,0,None,0,None,0,None,None,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,[],"-- created by Oraschemadoc Wed Jan 9 15:07:06 2019\n-- visit http://www.yarpen.cz/oraschemadoc/ for more info\n\n CREATE TABLE ""SPACEUSER"".""RHNSNAPSHOTCONFIGCHANNEL""\n (\t""SNAPSHOT_ID"" NUMBER NOT NULL ENABLE,\n\t""CONFIG_CHANNEL_ID"" NUMBER NOT NULL ENABLE,\n\t""CREATED"" TIMESTAMP (6) WITH LOCAL TIME ZONE DEFAULT (current_timestamp) NOT NULL ENABLE,\n\t""MODIFIED"" TIMESTAMP (6) WITH LOCAL TIME ZONE DEFAULT (current_timestamp) NOT NULL ENABLE\n ) ENABLE ROW MOVEMENT ;"
1,419753,1,0,NaN,NaN,NaN,NaN,None,Invalid expression / Unexpected token|,"HANNEL"" ADD CONSTRAINT ""RHN_SNAPSHOTCC_SID_FK"" FOREIGN KEY (""SNAPSHOT_ID"")\n\t REFERENCES ""SPACEUSER""|",.|,0,0,None,0,None,0,None,0,None,None,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,[],"ALTER TABLE ""SPACEUSER"".""RHNSNAPSHOTCONFIGCHANNEL"" ADD CONSTRAINT ""RHN_SNAPSHOTCC_SID_FK"" FOREIGN KEY (""SNAPSHOT_ID"")\n\t REFERENCES ""SPACEUSER"".""RHNSNAPSHOT"" (""ID"") ON DELETE CASCADE ENABLE;"
2,419753,2,0,NaN,NaN,NaN,NaN,None,Invalid expression / Unexpected token|,"ADD CONSTRAINT ""RHN_SNAPSHOTCC_CCID_FK"" FOREIGN KEY (""CONFIG_CHANNEL_ID"")\n\t REFERENCES ""SPACEUSER""|",.|,0,0,None,0,None,0,None,0,None,None,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,[],"ALTER TABLE ""SPACEUSER"".""RHNSNAPSHOTCONFIGCHANNEL"" ADD CONSTRAINT ""RHN_SNAPSHOTCC_CCID_FK"" FOREIGN KEY (""CONFIG_CHANNEL_ID"")\n\t REFERENCES ""SPACEUSER"".""RHNCONFIGCHANNEL"" (""ID"") ENABLE;"
3,419754,0,1,NaN,NaN,NaN,NaN,mysql,None,None,None,1,0,None,0,None,0,None,1,[RESERVAS_COVINOC],None,0,0,0,0,0,1,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"[{'Create': 1}, {'Table': 1}, {'Identifier': 1}]",CREATE DATABASE IF NOT EXISTS `reservas_covinoc` /*!40100 DEFAULT CHARACTER SET latin1 */;
4,419754,1,1,NaN,NaN,NaN,NaN,mysql,None,None,None,1,1,[RESERVAS_COVINOC],0,None,0,None,0,None,None,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"[{'Use': 1}, {'Table': 1}, {'Identifier': 1}]",USE `reservas_covinoc`;


## File-level SR

In [5]:
sqlglot_sr = sqlglot_ddf.groupby('parsed')['parsed'].count().compute()
sqlglot_sr = pd.DataFrame({'success':sqlglot_sr.index, 'count':[sqlglot_sr[sqlglot_sr.index == 0].sum(),sqlglot_sr[sqlglot_sr.index != 0].sum()]})
sqlglot_sr['percentage'] = sqlglot_sr['count']/filedetails_df['file_id'].count()
display(sqlglot_sr)

success_count = sqlglot_sr[sqlglot_sr.success == 1]['count'].sum()

[########################################] | 100% Completed | 547.45 ms


,success,count,percentage
0,0,281848,0.755309
1,1,91308,0.244691


### Per dialect

In [6]:
sqlglot_sr_ansi = sqlglot_ddf.groupby('parsed_none')['parsed_none'].count().compute()
sqlglot_sr_ansi = pd.DataFrame({'success':sqlglot_sr_ansi.index, 'count':[sqlglot_sr_ansi[sqlglot_sr_ansi.index == 0].sum(),sqlglot_sr_ansi[sqlglot_sr_ansi.index != 0].sum()]})
sqlglot_sr_ansi['percentage_full_dataset'] = sqlglot_sr_ansi['count']/filedetails_df['file_id'].count()
sqlglot_sr_ansi['percentage_parsed_sqlglot'] = sqlglot_sr_ansi['count']/success_count
sqlglot_sr_ansi['dialect'] = 'ANSI'

sqlglot_sr_postgres = sqlglot_ddf.groupby('parsed_postgres')['parsed_postgres'].count().compute()
sqlglot_sr_postgres = pd.DataFrame({'success':sqlglot_sr_postgres.index, 'count':[sqlglot_sr_postgres[sqlglot_sr_postgres.index == 0].sum(),sqlglot_sr_postgres[sqlglot_sr_postgres.index != 0].sum()]})
sqlglot_sr_postgres['percentage_full_dataset'] = sqlglot_sr_postgres['count']/filedetails_df['file_id'].count()
sqlglot_sr_postgres['percentage_parsed_sqlglot'] = sqlglot_sr_postgres['count']/success_count
sqlglot_sr_postgres['dialect'] = 'POSTGRES'

sqlglot_sr_mysql = sqlglot_ddf.groupby('parsed_mysql')['parsed_mysql'].count().compute()
sqlglot_sr_mysql = pd.DataFrame({'success':sqlglot_sr_mysql.index, 'count':[sqlglot_sr_mysql[sqlglot_sr_mysql.index == 0].sum(),sqlglot_sr_mysql[sqlglot_sr_mysql.index != 0].sum()]})
sqlglot_sr_mysql['percentage_full_dataset'] = sqlglot_sr_mysql['count']/filedetails_df['file_id'].count()
sqlglot_sr_mysql['percentage_parsed_sqlglot'] = sqlglot_sr_mysql['count']/success_count
sqlglot_sr_mysql['dialect'] = 'MYSQL'

sqlglot_sr_tsql = sqlglot_ddf.groupby('parsed_tsql')['parsed_tsql'].count().compute()
sqlglot_sr_tsql = pd.DataFrame({'success':sqlglot_sr_tsql.index, 'count':[sqlglot_sr_tsql[sqlglot_sr_tsql.index == 0].sum(),sqlglot_sr_tsql[sqlglot_sr_tsql.index != 0].sum()]})
sqlglot_sr_tsql['percentage_full_dataset'] = sqlglot_sr_tsql['count']/filedetails_df['file_id'].count()
sqlglot_sr_tsql['percentage_parsed_sqlglot'] = sqlglot_sr_tsql['count']/success_count
sqlglot_sr_tsql['dialect'] = 'TSQL'

merged_dialect_ddf = pd.concat([sqlglot_sr_ansi,sqlglot_sr_postgres,sqlglot_sr_mysql,sqlglot_sr_tsql],ignore_index=True)
display(merged_dialect_ddf[merged_dialect_ddf.success == 1])

[########################################] | 100% Completed | 567.00 ms
[########################################] | 100% Completed | 571.76 ms
[########################################] | 100% Completed | 573.70 ms
[########################################] | 100% Completed | 563.41 ms


,success,count,percentage_full_dataset,percentage_parsed_sqlglot,dialect
1,1,59938,0.160625,0.656438,ANSI
3,1,73850,0.197907,0.808801,POSTGRES
5,1,71112,0.190569,0.778815,MYSQL
7,1,63378,0.169843,0.694112,TSQL


## Comparison with pglast

In [ ]:
pglast_ddf = read_partial('pglast','file',PGLastSchema().file_level_schema.names)
sqlglot_ddf = read_partial('sqlglot','file',SQLGlotSchema().file_level_schema.names)

In [ ]:
pglast_sr = pglast_ddf.groupby('parsed_file')['parsed_file'].count().compute()
sqlglot_sr = sqlglot_ddf.groupby('parsed_postgres')['parsed_postgres'].count().compute()

pglast_sr = pd.DataFrame({'success':pglast_sr.index, 'count':[pglast_sr[pglast_sr.index == 0].sum(),pglast_sr[pglast_sr.index != 0].sum()]})
sqlglot_sr = pd.DataFrame({'success':sqlglot_sr.index, 'count':[sqlglot_sr[sqlglot_sr.index == 0].sum(),sqlglot_sr[sqlglot_sr.index != 0].sum()]})

pglast_sr['percentage'] = pglast_sr['count']/filedetails_df['file_id'].count()
sqlglot_sr['percentage'] = sqlglot_sr['count']/filedetails_df['file_id'].count()

pglast_sr['parser'] = 'PGLAST'
sqlglot_sr['parser'] = 'SQLGLOT_POSTGRES'

merged_dialect_ddf = pd.concat([pglast_sr,sqlglot_sr],ignore_index=True)
display(merged_dialect_ddf)

=> First issue: SQLGLOT with postgres dialect has almost 10k extra parsed files

Joining and writing down

In [ ]:
sqlglot_ddf = read_partial('sqlglot','file',[item for item in SQLGlotSchema().file_level_schema.names if "postgres" in item or "file_id" in item])
pglast_ddf = read_partial('pglast','file',PGLastSchema().file_level_schema.names)

# filtering
#pglast_ddf['p'] = pglast_ddf['parsed_file']
#sqlglot_ddf['p'] = sqlglot_ddf['parsed_postgres']
sqlglot_ddf = sqlglot_ddf[sqlglot_ddf.parsed_postgres == 1]
pglast_ddf = pglast_ddf[pglast_ddf.parsed_file == 1]

# renaming cols
sqlglot_ddf = sqlglot_ddf.rename(columns=dict(zip(sqlglot_ddf.columns, [item.replace("postgres","sqlglot") for item in sqlglot_ddf.columns])))
pglast_ddf = pglast_ddf.rename(columns=dict(zip(pglast_ddf.columns, [item.replace("pglast_file_id","file_id") for item in ["pglast_" + item for item in pglast_ddf.columns]])))

pglast_joined_sqlglot = dd.merge(
    sqlglot_ddf,
    pglast_ddf,
    on=['file_id'],
    how='outer',indicator=True,suffixes=['_sqlglot','_pglast'])

In [6]:
join_schema = pa.schema([])
for item in SQLGlotSchema().file_level_schema:
    if "postgres" in item.name or "file_id" in item.name:
        join_schema = join_schema.append(pa.field(item.name.replace("postgres","sqlglot"),item.type))
for item in PGLastSchema().file_level_schema:
    if item.name != 'file_id':
        join_schema = join_schema.append(pa.field("pglast_" + item.name,item.type))
join_schema = join_schema.append(pa.field("_merge",pa.string()))
pglast_joined_sqlglot.to_parquet('../out_new/join__parsed__pglast_sqlglot/',schema=join_schema,overwrite=True)

In [7]:
joined_ddf = dd.read_parquet('../out_new/join__parsed__pglast_sqlglot/', schema=join_schema, split_row_groups=True, calculate_divisions=True,engine='pyarrow')

In [8]:
joined_ddf.groupby('_merge')['_merge'].count().compute()
# "left_only" == sqlglot
# "right_only" == pglast

[########################################] | 100% Completed | 793.39 ms


_merge
both          40006
left_only     33844
right_only    23931
Name: _merge, dtype: int64

How many of the parsed files match for both?

40 006 only
aka 62% of those parsed by pglast

How many parsed files match on: parsed + number of extracted tables?

In [9]:
parsed_both_ddf = joined_ddf[joined_ddf['_merge'] == 'both']

# removing create seq stmt cause apparently it affects num tables
import ast
parsed_both_ddf['x'] = parsed_both_ddf.apply(
    lambda row:
    [item for item in ast.literal_eval(row['pglast_counter_str']) if 'CreateSeqStmt' in item],
    axis=1,
    meta=('x', 'string'))
parsed_both_ddf['pglast_num_create_seq'] = parsed_both_ddf.apply(
    lambda row:
    row['x'][0]['CreateSeqStmt'] if len(row['x']) > 0 else 0,
    axis=1,
    meta=('pglast_num_create_seq', 'float'))

parsed_both_ddf['sqlglot_nt'] = parsed_both_ddf['sqlglot_num_distinct_tables']
parsed_both_ddf['pglast_nt'] = parsed_both_ddf['pglast_num_distinct_tables'] - parsed_both_ddf['pglast_num_create_seq']

parsed_both_ddf[parsed_both_ddf.sqlglot_nt == parsed_both_ddf.pglast_nt]['file_id'].count().compute()

[########################################] | 100% Completed | 6.67 sms


38961

Match on num columns

In [10]:
parsed_both_ddf[parsed_both_ddf.pglast_num_distinct_columns == parsed_both_ddf.sqlglot_num_distinct_columns]['file_id'].count().compute()

[########################################] | 100% Completed | 6.29 ss


39943

Match on num tables + num columns

In [11]:
parsed_both_ddf[(parsed_both_ddf.pglast_num_distinct_columns == parsed_both_ddf.sqlglot_num_distinct_columns) &
                (parsed_both_ddf.sqlglot_nt == parsed_both_ddf.pglast_nt)]['file_id'].count().compute()

[########################################] | 100% Completed | 5.96 ss


38943

Match on constraints

notnull: 38532
unique: 34763
primary: 25282
foreign: 37411
all: 19085

In [12]:
parsed_both_ddf[(parsed_both_ddf.sqlglot_num_ctr_notnull == parsed_both_ddf.pglast_num_ctr_notnull) &
                (parsed_both_ddf.sqlglot_num_ctr_unique == parsed_both_ddf.pglast_num_ctr_unique) &
                (parsed_both_ddf.sqlglot_num_ctr_primary == parsed_both_ddf.pglast_num_ctr_primary) &
                (parsed_both_ddf.sqlglot_num_ctr_foreign == parsed_both_ddf.pglast_num_ctr_foreign)
                ]['file_id'].count().compute()

[########################################] | 100% Completed | 7.33 sms


19085

Match on table list

In [13]:
parsed_both_ddf_tl = parsed_both_ddf.dropna(subset=['sqlglot_table_list','pglast_table_list'])
parsed_both_ddf_tl['sqlglot_tl'] = parsed_both_ddf_tl.apply(
    lambda row:
    "|".join(str(x) for x in sorted(row['sqlglot_table_list'])),
    axis=1,
    meta=('sqlglot_tl', 'string'))
parsed_both_ddf_tl['pglast_tl'] = parsed_both_ddf_tl.apply(
    lambda row:
    "|".join(str(x) for x in sorted(row['pglast_table_list'])),
    axis=1,
    meta=('pglast_tl', 'string'))

parsed_both_ddf_tl[parsed_both_ddf_tl.pglast_tl == parsed_both_ddf_tl.sqlglot_tl]['file_id'].count().compute()

[########################################] | 100% Completed | 8.75 sms


35927

Match on column list

In [ ]:
parsed_both_ddf_cl = parsed_both_ddf.dropna(subset=['sqlglot_column_list','pglast_columns_list'])
parsed_both_ddf_cl['sqlglot_cl'] = parsed_both_ddf_cl.apply(
    lambda row:
    "|".join(str(x) for x in sorted(row['sqlglot_column_list'])),
    axis=1,
    meta=('sqlglot_cl', 'string'))
parsed_both_ddf_cl['pglast_cl'] = parsed_both_ddf_cl.apply(
    lambda row:
    "|".join(str(x) for x in sorted(row['pglast_columns_list'])),
    axis=1,
    meta=('pglast_cl', 'string'))

parsed_both_ddf_cl[parsed_both_ddf_cl.pglast_cl == parsed_both_ddf_cl.sqlglot_cl]['file_id'].count().compute()

# Error Analysis

In [4]:
sqlglot_ddf = read_partial('sqlglot','file',SQLGlotSchema().file_level_schema.names)

In [5]:
sqlglot_ddf.head()

[########################################] | 100% Completed | 117.80 ms


,file_id,file_path,errors_at_read,parsed,parsed_none,parsed_postgres,parsed_mysql,parsed_tsql,dialect,parse_error,file_parse_error_start_context,file_parse_error_highlight,num_statements,num_distinct_tables,table_list,num_distinct_columns,column_list,num_distinct_schemas,schema_list,num_distinct_dbs,db_list,view_list,num_constraints,num_ctr_notnull,num_ctr_unique,num_ctr_primary,num_ctr_foreign,comment_count,num_drop_stmt,num_drop_like_stmt,num_create_stmt,num_create_like_stmt,num_insert_stmt,num_insert_like_stmt,num_alter_stmt,num_alter_like_stmt,num_select_stmt,num_view_stmt,num_truncate_stmt,num_update_stmt,num_comment_stmt,num_delete_stmt,num_set_like_stmt,num_execute_like_stmt,num_index_like_stmt,num_transaction_like_stmt,num_command_stmt,num_command_alter_stmt,counter_str,postgres_num_statements,postgres_num_distinct_tables,postgres_table_list,postgres_num_distinct_columns,postgres_column_list,postgres_num_distinct_schemas,postgres_schema_list,postgres_num_distinct_dbs,postgres_db_list,postgres_view_list,postgres_num_constraints,postgres_num_ctr_notnull,postgres_num_ctr_unique,postgres_num_ctr_primary,postgres_num_ctr_foreign,postgres_comment_count,postgres_num_drop_stmt,postgres_num_drop_like_stmt,postgres_num_create_stmt,postgres_num_create_like_stmt,postgres_num_insert_stmt,postgres_num_insert_like_stmt,postgres_num_alter_stmt,postgres_num_alter_like_stmt,postgres_num_select_stmt,postgres_num_view_stmt,postgres_num_truncate_stmt,postgres_num_update_stmt,postgres_num_comment_stmt,postgres_num_delete_stmt,postgres_num_set_like_stmt,postgres_num_execute_like_stmt,postgres_num_index_like_stmt,postgres_num_transaction_like_stmt,postgres_num_command_stmt,postgres_num_command_alter_stmt,postgres_counter_str
0,413760,../sqlfiles/413760_RRInitialQuery.sql,0,0,0,0,0,0,None,Expecting (|Expecting )|Invalid expression / Unexpected token|,"ng INT CHECK(Rating >= 0 and Rating <= 5) NOT NULL,\n Note TEXT,\n RestaurantId INT FOREIGN KEY |K(Rating >= 0 and Rating <= 5) NOT NULL,\n Note TEXT,\n RestaurantId INT FOREIGN KEY REFERENCES |",REFERENCES|Restaurants|,0,0,None,0,None,0,None,0,None,None,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,[],0,0,None,0,None,0,None,0,None,None,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,None
1,413761,../sqlfiles/413761_dam.spe.req_create_altibase.sql,0,0,0,0,0,0,None,Expecting )|Invalid expression / Unexpected token|,CMMNCLCODE(CL_CODE) ON DELETE CASCADE\n);\n\nCREATE INDEX COMTCCMMNCODE_i01 ON COMTCCMMNCODE\n(CL_CODE |,ASC|,0,0,None,0,None,0,None,0,None,None,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,[],0,0,None,0,None,0,None,0,None,None,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,None
2,413762,../sqlfiles/413762_V002_1__create_item_table.sql,0,1,1,1,1,1,None,None,None,None,1,2,"[ORDERS, ITEM]",7,"[COLOR, SIZE, ORDER_ID, NAME, AMOUNT, ID, CURRENCY]",0,None,0,None,None,6,5,0,1,1,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"[{'Create': 1}, {'Schema': 1}, {'Table': 1}, {'ColumnDef': 7}, {'PrimaryKey': 1}, {'ForeignKey': 1}, {'Identifier': 12}, {'DataType': 7}, {'ColumnConstraint': 6}, {'Reference': 1}, {'GeneratedAsIdentityColumnConstraint': 1}, {'Literal': 4}, {'NotNullColumnConstraint': 5}]",1,2,"[ORDERS, ITEM]",7,"[COLOR, SIZE, ORDER_ID, NAME, AMOUNT, ID, CURRENCY]",0,None,0,None,None,6,5,0,1,1,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,None
3,413764,../sqlfiles/413764_V001_01__initial%20db%20creation.sql,0,0,0,0,0,0,None,Invalid expression / Unexpected token|,ULL\n);\n\nCREATE TABLE HIBERNATE_SEQUENCE (\n next_val BIGINT(20) NULL\n);\n\nALTER TABLE USER_AUTHORITY |,ADD|,0,0,None,0,None,0,None,0,None,None,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,[],0,0,None,0,None,0,None,0,None,None,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,None
4,413765,../sqlfiles/413765_DEPARTMENTS.sql,0,0,0,0,0,0,None,Expecting )|Expecting )|Invalid expression / Unexpected token|,"REATE TABLE hr_prod.departments (\n department_id NUMBER(4) NOT NULL,\n department_name VARCHAR2(30 |",B

In [6]:
sqlglot_ddf.groupby('parsed')['parsed'].count().compute()

[########################################] | 100% Completed | 663.10 ms


parsed
0    281848
1    91308 
Name: parsed, dtype: int64

### stmt level

In [7]:
sqlparser_details_ddf = read_partial('sqlglot','stmt',SQLGlotSchema().statement_list_sch.names)
sqlparser_details_ddf.groupby('parsed')['parsed'].count().compute()

[########################################] | 100% Completed | 1.07 sms


parsed
0    3676582 
1    20950924
Name: parsed, dtype: int64

### back to errs

In [34]:
not_parsed = sqlglot_ddf[sqlglot_ddf.parsed == 0][['file_id','file_path','parse_error','file_parse_error_start_context', 'file_parse_error_highlight']]

# 57 979 DEFAULT -> 56 829 DEFAULT + ENGINE
not_parsed[(not_parsed.file_parse_error_highlight.str.upper().str.contains('DEFAULT')) &
           (not_parsed.file_parse_error_start_context.str.upper().str.contains('ENGINE'))
           ].count().compute()

[########################################] | 100% Completed | 1.73 sms


file_id                           56829
file_path                         56829
parse_error                       56829
file_parse_error_start_context    56829
file_parse_error_highlight        56829
dtype: int64

In [41]:
# UNSIGNED upper -> 35 992 -> also INT: 35 863
not_parsed[(not_parsed.file_parse_error_highlight.str.upper().str.contains('UNSIGNED')) &
           (not_parsed.file_parse_error_start_context.str.upper().str.contains('INT',regex=False))
           ].count().compute()

[########################################] | 100% Completed | 1.99 sms


file_id                           35863
file_path                         35863
parse_error                       35863
file_parse_error_start_context    35863
file_parse_error_highlight        35863
dtype: int64

In [42]:
not_parsed[(not_parsed.file_parse_error_highlight.str.upper().str.contains('KEY'))].head()

[########################################] | 100% Completed | 128.07 ms


,file_id,file_path,parse_error,file_parse_error_start_context,file_parse_error_highlight
100,413962,../sqlfiles/413962_roomReser_dump.sql,Expecting )|Invalid expression / Unexpected token|,"ult 'None',\n `booking_id` int(11) NOT NULL auto_increment,\n PRIMARY KEY (`booking_id`),\n UNIQUE |",KEY|
114,413988,../sqlfiles/413988_UserTravelMonths.sql,Expecting )|Invalid expression / Unexpected token|,"D),\n FOREIGN KEY (UserEmailAddress) REFERENCES Users(UserEmailAddress) ON DELETE CASCADE,\n UNIQUE |",KEY|
176,414126,../sqlfiles/414126_efhm_db_cocoon_role_allow_assign.sql,Expecting )|Invalid expression / Unexpected token|,"t NOT NULL DEFAULT '0',\n `allowassign` bigint NOT NULL DEFAULT '0',\n PRIMARY KEY (`id`),\n UNIQUE |",KEY|
182,414139,../sqlfiles/414139_vms.sql,Expecting )|Invalid expression / Unexpected token|,"`id` int(11) NOT NULL AUTO_INCREMENT,\n `name` varchar(80) NOT NULL,\n PRIMARY KEY (`id`),\n UNIQUE |",KEY|
218,414195,../sqlfiles/414195_portal_professor.sql,Expecting )|Invalid expression / Unexpected token|,"_dep` varchar(20) NOT NULL,\n `pr_res_num` varchar(14) NOT NULL,\n PRIMARY KEY (`pr_num`),\n UNIQUE |",KEY|


In [19]:
errcounts = not_parsed['file_parse_error_highlight'].value_counts().compute()
err_df = pd.DataFrame({'err':errcounts.index, 'cnt':errcounts.values})

[########################################] | 100% Completed | 1.10 sms


In [22]:
err_df.head(50)

,err,cnt
0,DEFAULT|,56840
1,unsigned|,22177
2,[|,18082
3,UNSIGNED|,13806
4,KEY|,10614
5,#|,9393
6,CREATE|,7186
7,ON|UPDATE|,5053
8,.|,4876
9,USING|)|,4394


# NEW error analysis

## Read and Prep -> SCHEMA HERE
also includes more useful error calculation

In [18]:
not_parsed_schema = pa.schema([])
not_parsed_schema = not_parsed_schema.append(pa.field("file_id",pa.string()))
not_parsed_schema = not_parsed_schema.append(pa.field("statement_id",pa.int64()))
not_parsed_schema = not_parsed_schema.append(pa.field("parsed",pa.int64()))
not_parsed_schema = not_parsed_schema.append(pa.field("parse_error",pa.string()))
not_parsed_schema = not_parsed_schema.append(pa.field("file_parse_error_start_context",pa.string()))
not_parsed_schema = not_parsed_schema.append(pa.field("file_parse_error_highlight",pa.string()))


not_parsed_schema = not_parsed_schema.append(pa.field("possible_tsql_1",pa.int64()))
not_parsed_schema = not_parsed_schema.append(pa.field("possible_tsql_2",pa.int64()))
not_parsed_schema = not_parsed_schema.append(pa.field("possible_tsql_3",pa.int64()))

not_parsed_schema = not_parsed_schema.append(pa.field("possible_orasql_1",pa.int64()))
not_parsed_schema = not_parsed_schema.append(pa.field("possible_orasql_2",pa.int64()))
not_parsed_schema = not_parsed_schema.append(pa.field("possible_orasql_3",pa.int64()))

not_parsed_schema = not_parsed_schema.append(pa.field("possible_mysql_1",pa.int64()))
not_parsed_schema = not_parsed_schema.append(pa.field("possible_mysql_2",pa.int64()))
not_parsed_schema = not_parsed_schema.append(pa.field("possible_mysql_3",pa.int64()))

not_parsed_schema = not_parsed_schema.append(pa.field("original",pa.string()))

In [63]:
sqlglot_details_ddf = read_partial('sqlglot','stmt',SQLGlotSchema().statement_list_sch.names)
sqlglot_details_ddf.groupby("parsed")['parsed'].count().compute(scheduler='processes')

[########################################] | 100% Completed | 3.35 ss


parsed
0    3676582 
1    20950924
Name: parsed, dtype: int64

In [64]:
sqlglot_details_ddf[sqlglot_details_ddf.parsed == 1]['file_id'].nunique().compute(scheduler='processes')

[########################################] | 100% Completed | 6.02 ss


328704

In [65]:
sqlglot_details_ddf[sqlglot_details_ddf.parsed == 0]['file_id'].nunique().compute(scheduler='processes')

[########################################] | 100% Completed | 5.23 ss


281282

In [21]:
not_parsed = sqlglot_details_ddf[sqlglot_details_ddf.parsed == 0]

In [22]:
import re
not_parsed['possible_tsql_1'] = not_parsed.apply(
    lambda row:
    len(re.findall('\[([a-z0-9A-Z])+\]', row['file_parse_error_start_context'])), 
    axis=1,
    meta=('possible_tsql_1', 'int')
)
not_parsed['possible_tsql_2'] = not_parsed.apply(
    lambda row:
    len(re.findall('\[([a-z0-9A-Z])+\]', row['file_parse_error_highlight'])), 
    axis=1,
    meta=('possible_tsql_2', 'int')
)
not_parsed['possible_tsql_3'] = not_parsed.apply(
    lambda row:
    1 if '[' in row['file_parse_error_highlight'] or ']' in row['file_parse_error_highlight'] else 0,
    axis=1,
    meta=('possible_tsql_3', 'int')
)


not_parsed['possible_orasql_1'] = not_parsed.apply(
    lambda row:
    len(re.findall('\"([a-z0-9A-Z])+\"', row['file_parse_error_start_context'])), 
    axis=1,
    meta=('possible_orasql_1', 'int')
)
not_parsed['possible_orasql_2'] = not_parsed.apply(
    lambda row:
    len(re.findall('\"([a-z0-9A-Z])+\"', row['file_parse_error_highlight'])), 
    axis=1,
    meta=('possible_orasql_2', 'int')
)
not_parsed['possible_orasql_3'] = not_parsed.apply(
    lambda row:
    1 if '"' in row['file_parse_error_highlight'] else 0,
    axis=1,
    meta=('possible_orasql_3', 'int')
)

not_parsed['possible_mysql_1'] = not_parsed.apply(
    lambda row:
    len(re.findall('`([a-z0-9A-Z])+`', row['file_parse_error_start_context'])), 
    axis=1,
    meta=('possible_mysql_1', 'int')
)
not_parsed['possible_mysql_2'] = not_parsed.apply(
    lambda row:
    len(re.findall('`([a-z0-9A-Z])+`', row['file_parse_error_highlight'])), 
    axis=1,
    meta=('possible_mysql_2', 'int')
)
not_parsed['possible_mysql_3'] = not_parsed.apply(
    lambda row:
    1 if '`' in row['file_parse_error_highlight'] else 0,
    axis=1,
    meta=('possible_mysql_3', 'int')
)

not_parsed = not_parsed[not_parsed_schema.names]

In [24]:
not_parsed.to_parquet('../out_new/sqlglot_unparsed_stmts_only/',schema=not_parsed_schema,overwrite=True)

[########################################] | 100% Completed | 313.98 s


In [48]:
not_parsed_only = dd.read_parquet('../out_new/sqlglot_unparsed_stmts_only/', columns = not_parsed_schema.names ,schema=not_parsed_schema, split_row_groups=True, calculate_divisions=True,engine='pyarrow')

In [49]:
not_parsed_only['file_id'].count().compute(scheduler='processes')

[########################################] | 100% Completed | 3.45 ss


3676582

In [50]:
import sqlglot
def get_good_error_1(row):
    parsed_statements = None
    _dialect = None
    if row['possible_tsql_1'] == 1 or row['possible_tsql_2'] == 1 or row['possible_tsql_3'] == 1:
        _dialect = "tsql"
    elif row['possible_orasql_1'] == 1 or row['possible_orasql_2'] == 1 or row['possible_orasql_3'] == 1:
        _dialect = "oracle"
    elif row['possible_mysql_1'] == 1 or row['possible_mysql_2'] == 1 or row['possible_mysql_3'] == 1:
        _dialect = "mysql"


    if "TIMEOUT_ERROR" in row['parse_error']:
        _dialect = None

    if _dialect is not None:
        _result = {}
        try:
            parsed_statements = sqlglot.parse(row['original'],read=_dialect,error_level=sqlglot.ErrorLevel.RAISE)
        except sqlglot.errors.ParseError as e:
            _result["parse_error"] = ''
            _result["file_parse_error_start_context"] = ''
            _result["file_parse_error_highlight"] = ''
            for idx, err in enumerate(e.errors):
                if idx > 0:
                    if err['highlight'] != e.errors[idx-1]['highlight']:
                        _result["file_parse_error_start_context"] += err['start_context'][0:200] + '|'
                        _result["file_parse_error_highlight"] += err['highlight'][0:200] + '|'
                    else:
                        _result["parse_error"] += err['description'][0:200] + '|'
                else:
                    _result["parse_error"] += err['description'][0:200] + '|'
                    
                    _result["file_parse_error_start_context"] += err['start_context'][0:200] + '|'
                    _result["file_parse_error_highlight"] += err['highlight'][0:200] + '|'

            _result["parse_error"] = _result["parse_error"][0:600]
            _result["file_parse_error_start_context"] = _result["file_parse_error_start_context"][0:400]
            _result["file_parse_error_highlight"] = _result["file_parse_error_highlight"][0:200]
        except Exception as e:
            _result['parse_error'] = 'NON_PARSER_ERROR|' + str(e)[0:300]
            _result['file_parse_error_start_context'] = ''
            _result['file_parse_error_highlight'] = ''
    else:
        _result = {}
        _result["parse_error"] = row['parse_error']
        _result["file_parse_error_start_context"] = row['file_parse_error_start_context']
        _result["file_parse_error_highlight"] = row['file_parse_error_highlight']
    
    return _result["parse_error"]#,_result["file_parse_error_start_context"],_result["file_parse_error_highlight"]

def get_good_error_2(row):
    parsed_statements = None
    _dialect = None
    if row['possible_tsql_1'] == 1 or row['possible_tsql_2'] == 1 or row['possible_tsql_3'] == 1:
        _dialect = "tsql"
    elif row['possible_orasql_1'] == 1 or row['possible_orasql_2'] == 1 or row['possible_orasql_3'] == 1:
        _dialect = "oracle"
    elif row['possible_mysql_1'] == 1 or row['possible_mysql_2'] == 1 or row['possible_mysql_3'] == 1:
        _dialect = "mysql"

    if "TIMEOUT_ERROR" in row['parse_error']:
        _dialect = None

    if _dialect is not None:
        _result = {}
        try:
            parsed_statements = sqlglot.parse(row['original'],read=_dialect,error_level=sqlglot.ErrorLevel.RAISE)
        except sqlglot.errors.ParseError as e:
            _result["parse_error"] = ''
            _result["file_parse_error_start_context"] = ''
            _result["file_parse_error_highlight"] = ''
            for idx, err in enumerate(e.errors):
                if idx > 0:
                    if err['highlight'] != e.errors[idx-1]['highlight']:
                        _result["file_parse_error_start_context"] += err['start_context'][0:200] + '|'
                        _result["file_parse_error_highlight"] += err['highlight'][0:200] + '|'
                    else:
                        _result["parse_error"] += err['description'][0:200] + '|'
                else:
                    _result["parse_error"] += err['description'][0:200] + '|'
                    
                    _result["file_parse_error_start_context"] += err['start_context'][0:200] + '|'
                    _result["file_parse_error_highlight"] += err['highlight'][0:200] + '|'

            _result["parse_error"] = _result["parse_error"][0:600]
            _result["file_parse_error_start_context"] = _result["file_parse_error_start_context"][0:400]
            _result["file_parse_error_highlight"] = _result["file_parse_error_highlight"][0:200]
        except Exception as e:
            _result['parse_error'] = 'NON_PARSER_ERROR|' + str(e)[0:300]
            _result['file_parse_error_start_context'] = ''
            _result['file_parse_error_highlight'] = ''
    else:
        _result = {}
        _result["parse_error"] = row['parse_error']
        _result["file_parse_error_start_context"] = row['file_parse_error_start_context']
        _result["file_parse_error_highlight"] = row['file_parse_error_highlight']
    
    return _result["file_parse_error_start_context"]

def get_good_error_3(row):
    parsed_statements = None
    _dialect = None
    if row['possible_tsql_1'] == 1 or row['possible_tsql_2'] == 1 or row['possible_tsql_3'] == 1:
        _dialect = "tsql"
    elif row['possible_orasql_1'] == 1 or row['possible_orasql_2'] == 1 or row['possible_orasql_3'] == 1:
        _dialect = "oracle"
    elif row['possible_mysql_1'] == 1 or row['possible_mysql_2'] == 1 or row['possible_mysql_3'] == 1:
        _dialect = "mysql"

    if "TIMEOUT_ERROR" in row['parse_error']:
        _dialect = None

    if _dialect is not None:
        _result = {}
        try:
            parsed_statements = sqlglot.parse(row['original'],read=_dialect,error_level=sqlglot.ErrorLevel.RAISE)
        except sqlglot.errors.ParseError as e:
            _result["parse_error"] = ''
            _result["file_parse_error_start_context"] = ''
            _result["file_parse_error_highlight"] = ''
            for idx, err in enumerate(e.errors):
                if idx > 0:
                    if err['highlight'] != e.errors[idx-1]['highlight']:
                        _result["file_parse_error_start_context"] += err['start_context'][0:200] + '|'
                        _result["file_parse_error_highlight"] += err['highlight'][0:200] + '|'
                    else:
                        _result["parse_error"] += err['description'][0:200] + '|'
                else:
                    _result["parse_error"] += err['description'][0:200] + '|'
                    
                    _result["file_parse_error_start_context"] += err['start_context'][0:200] + '|'
                    _result["file_parse_error_highlight"] += err['highlight'][0:200] + '|'

            _result["parse_error"] = _result["parse_error"][0:600]
            _result["file_parse_error_start_context"] = _result["file_parse_error_start_context"][0:400]
            _result["file_parse_error_highlight"] = _result["file_parse_error_highlight"][0:200]
        except Exception as e:
            _result['parse_error'] = 'NON_PARSER_ERROR|' + str(e)[0:300]
            _result['file_parse_error_start_context'] = ''
            _result['file_parse_error_highlight'] = ''
    else:
        _result = {}
        _result["parse_error"] = row['parse_error']
        _result["file_parse_error_start_context"] = row['file_parse_error_start_context']
        _result["file_parse_error_highlight"] = row['file_parse_error_highlight']
    
    return _result["file_parse_error_highlight"]

In [51]:
not_parsed_only['good_parse_error'] = not_parsed_only.apply(
    get_good_error_1,
    axis=1,
    meta=('good_parse_error', 'string')
)

not_parsed_only['good_file_parse_error_start_context'] = not_parsed_only.apply(
    get_good_error_2,
    axis=1,
    meta=('good_file_parse_error_start_context', 'string')
)

not_parsed_only['good_file_parse_error_highlight'] = not_parsed_only.apply(
    get_good_error_3,
    axis=1,
    meta=('good_file_parse_error_highlight', 'string')
)

In [52]:
not_parsed_schema = pa.schema([])
not_parsed_schema = not_parsed_schema.append(pa.field("file_id",pa.string()))
not_parsed_schema = not_parsed_schema.append(pa.field("statement_id",pa.int64()))
not_parsed_schema = not_parsed_schema.append(pa.field("parsed",pa.int64()))
not_parsed_schema = not_parsed_schema.append(pa.field("parse_error",pa.string()))
not_parsed_schema = not_parsed_schema.append(pa.field("file_parse_error_start_context",pa.string()))
not_parsed_schema = not_parsed_schema.append(pa.field("file_parse_error_highlight",pa.string()))


not_parsed_schema = not_parsed_schema.append(pa.field("possible_tsql_1",pa.int64()))
not_parsed_schema = not_parsed_schema.append(pa.field("possible_tsql_2",pa.int64()))
not_parsed_schema = not_parsed_schema.append(pa.field("possible_tsql_3",pa.int64()))

not_parsed_schema = not_parsed_schema.append(pa.field("possible_orasql_1",pa.int64()))
not_parsed_schema = not_parsed_schema.append(pa.field("possible_orasql_2",pa.int64()))
not_parsed_schema = not_parsed_schema.append(pa.field("possible_orasql_3",pa.int64()))

not_parsed_schema = not_parsed_schema.append(pa.field("possible_mysql_1",pa.int64()))
not_parsed_schema = not_parsed_schema.append(pa.field("possible_mysql_2",pa.int64()))
not_parsed_schema = not_parsed_schema.append(pa.field("possible_mysql_3",pa.int64()))

not_parsed_schema = not_parsed_schema.append(pa.field("original",pa.string()))

not_parsed_schema = not_parsed_schema.append(pa.field("good_parse_error",pa.string()))
not_parsed_schema = not_parsed_schema.append(pa.field("good_file_parse_error_start_context",pa.string()))
not_parsed_schema = not_parsed_schema.append(pa.field("good_file_parse_error_highlight",pa.string()))


In [53]:
#not_parsed_only.to_parquet('../out_new/sqlglot_unparsed_stmts_only_good_errors/',schema=not_parsed_schema,overwrite=True,
compute_kwargs={'scheduler':'processes'})

[########################################] | 100% Completed | 48m 38s


In [47]:
not_parsed_only[
    (not_parsed_only.file_id == '419789') | 
    (not_parsed_only.file_id == '419754') | 
    (not_parsed_only.file_id == '419753')].head(10)

# 419789 for tsql
# 419754 for mysql
# 419753 for ora sql

[########################################] | 100% Completed | 58.45 s


c:\Users\geacu\anaconda3\envs\windows\lib\site-packages\dask\dataframe\core.py:7893: UserWarning: Insufficient elements for `head`. 10 elements requested, only 6 elements available. Try passing larger `npartitions` to `head`.
  warnings.warn(


,file_id,statement_id,parsed,parse_error,file_parse_error_start_context,file_parse_error_highlight,possible_tsql_1,possible_tsql_2,possible_tsql_3,possible_orasql_1,possible_orasql_2,possible_orasql_3,possible_mysql_1,possible_mysql_2,possible_mysql_3,original,good_parse_error,good_file_parse_error_start_context,good_file_parse_error_highlight
0,419753,0,0,Expecting )|Invalid expression / Unexpected token|,"more info\n\n CREATE TABLE ""SPACEUSER"".""RHNSNAPSHOTCONFIGCHANNEL""\n (\t""SNAPSHOT_ID"" NUMBER NOT NULL |",ENABLE|,0,0,0,2,0,0,0,0,0,"-- created by Oraschemadoc Wed Jan 9 15:07:06 2019\n-- visit http://www.yarpen.cz/oraschemadoc/ for more info\n\n CREATE TABLE ""SPACEUSER"".""RHNSNAPSHOTCONFIGCHANNEL""\n (\t""SNAPSHOT_ID"" NUMBER NOT NULL ENABLE,\n\t""CONFIG_CHANNEL_ID"" NUMBER NOT NULL ENABLE,\n\t""CREATED"" TIMESTAMP (6) WITH LOCAL TIME ZONE DEFAULT (current_timestamp) NOT NULL ENABLE,\n\t""MODIFIED"" TIMESTAMP (6) WITH LOCAL TIME ZONE DEFAULT (current_timestamp) NOT NULL ENABLE\n ) ENABLE ROW MOVEMENT ;",Expecting )|Invalid expression / Unexpected token|,"more info\n\n CREATE TABLE ""SPACEUSER"".""RHNSNAPSHOTCONFIGCHANNEL""\n (\t""SNAPSHOT_ID"" NUMBER NOT NULL |",ENABLE|
1,419753,1,0,Invalid expression / Unexpected token|,"HANNEL"" ADD CONSTRAINT ""RHN_SNAPSHOTCC_SID_FK"" FOREIGN KEY (""SNAPSHOT_ID"")\n\t REFERENCES ""SPACEUSER""|",.|,0,0,0,1,0,0,0,0,0,"ALTER TABLE ""SPACEUSER"".""RHNSNAPSHOTCONFIGCHANNEL"" ADD CONSTRAINT ""RHN_SNAPSHOTCC_SID_FK"" FOREIGN KEY (""SNAPSHOT_ID"")\n\t REFERENCES ""SPACEUSER"".""RHNSNAPSHOT"" (""ID"") ON DELETE CASCADE ENABLE;",Invalid expression / Unexpected token|,"HANNEL"" ADD CONSTRAINT ""RHN_SNAPSHOTCC_SID_FK"" FOREIGN KEY (""SNAPSHOT_ID"")\n\t REFERENCES ""SPACEUSER""|",.|
2,419753,2,0,Invalid expression / Unexpected token|,"ADD CONSTRAINT ""RHN_SNAPSHOTCC_CCID_FK"" FOREIGN KEY (""CONFIG_CHANNEL_ID"")\n\t REFERENCES ""SPACEUSER""|",.|,0,0,0,1,0,0,0,0,0,"ALTER TABLE ""SPACEUSER"".""RHNSNAPSHOTCONFIGCHANNEL"" ADD CONSTRAINT ""RHN_SNAPSHOTCC_CCID_FK"" FOREIGN KEY (""CONFIG_CHANNEL_ID"")\n\t REFERENCES ""SPACEUSER"".""RHNCONFIGCHANNEL"" (""ID"") ENABLE;",Invalid expression / Unexpected token|,"ADD CONSTRAINT ""RHN_SNAPSHOTCC_CCID_FK"" FOREIGN KEY (""CONFIG_CHANNEL_ID"")\n\t REFERENCES ""SPACEUSER""|",.|
3,419754,15,0,Expecting )|Invalid expression / Unexpected token|,"LL,\n `hasta` datetime DEFAULT NULL,\n `Status` int(11) DEFAULT NULL,\n PRIMARY KEY (`id`),\n KEY `f|",k_reservas_1_idx`|,0,0,0,0,0,0,3,0,1,"CREATE TABLE `reservas` (\n `id` int(11) NOT NULL AUTO_INCREMENT,\n `id_empleado` int(11) DEFAULT NULL,\n `id_servicio` int(11) DEFAULT NULL,\n `id_sucursal` int(11) DEFAULT NULL,\n `desde` datetime DEFAULT NULL,\n `hasta` datetime DEFAULT NULL,\n `Status` int(11) DEFAULT NULL,\n PRIMARY KEY (`id`),\n KEY `fk_reservas_1_idx` (`id_empleado`),\n KEY `fk_servicio_idx` (`id_servicio`),\n KEY `fk_sucursal_idx` (`id_sucursal`),\n CONSTRAINT `fk_empleado` FOREIGN KEY (`id_empleado`) REFERENCES `empleados` (`id`) ON DELETE NO ACTION ON UPDATE NO ACTION,\n CONSTRAINT `fk_servicio` FOREIGN KEY (`id_servicio`) REFERENCES `servicios` (`id`) ON DELETE NO ACTION ON UPDATE NO ACTION,\n CONSTRAINT `fk_sucursal` FOREIGN KEY (`id_sucursal`) REFERENCES `sucursales` (`id`) ON DELETE NO ACTION ON UPDATE NO ACTION\n) ENGINE=InnoDB AUTO_INCREMENT=13 DEFAULT CHARSET=latin1;",Expecting )|Invalid expression / Unexpected token|,"LL,\n `hasta` datetime DEFAULT NULL,\n `Status` int(11) DEFAULT NULL,\n PRIMARY KEY (`id`),\n KEY `f|",k_reservas_1_idx`|
4,419754,17,0,Invalid expression / Unexpected token|,--\n-- Dumping data for table `reservas`\n--\n\nLOCK TABLES `r|,eservas`|,0,0,0,0,0,0,1,0,1,--\n-- Dumping data for table `reservas`\n--\n\nLOCK TABLES `reservas` WRITE;,Invalid expression / Unexpected token|,--\n-- Dumping data for table `reservas`\n--\n\nLOCK TABLES `r|,eservas`|
123,419789,0,0,"Expected table name but got <Token token_type: TokenType.L_BRACKET, text: [, line: 1, col: 14, comments: []>|Required ke

## Read with new errors -> SCHEMA HERE

In [58]:
def get_stmt_and_file_count(dask_df):
    x = dask_df['file_id'].nunique().compute(scheduler='processes')
    y = dask_df['file_id'].count().compute(scheduler='processes')
    return y,x

In [78]:
frac = 10/3676582

In [54]:
not_parsed_schema = pa.schema([])
not_parsed_schema = not_parsed_schema.append(pa.field("file_id",pa.string()))
not_parsed_schema = not_parsed_schema.append(pa.field("statement_id",pa.int64()))
not_parsed_schema = not_parsed_schema.append(pa.field("parsed",pa.int64()))
not_parsed_schema = not_parsed_schema.append(pa.field("parse_error",pa.string()))
not_parsed_schema = not_parsed_schema.append(pa.field("file_parse_error_start_context",pa.string()))
not_parsed_schema = not_parsed_schema.append(pa.field("file_parse_error_highlight",pa.string()))


not_parsed_schema = not_parsed_schema.append(pa.field("possible_tsql_1",pa.int64()))
not_parsed_schema = not_parsed_schema.append(pa.field("possible_tsql_2",pa.int64()))
not_parsed_schema = not_parsed_schema.append(pa.field("possible_tsql_3",pa.int64()))

not_parsed_schema = not_parsed_schema.append(pa.field("possible_orasql_1",pa.int64()))
not_parsed_schema = not_parsed_schema.append(pa.field("possible_orasql_2",pa.int64()))
not_parsed_schema = not_parsed_schema.append(pa.field("possible_orasql_3",pa.int64()))

not_parsed_schema = not_parsed_schema.append(pa.field("possible_mysql_1",pa.int64()))
not_parsed_schema = not_parsed_schema.append(pa.field("possible_mysql_2",pa.int64()))
not_parsed_schema = not_parsed_schema.append(pa.field("possible_mysql_3",pa.int64()))

not_parsed_schema = not_parsed_schema.append(pa.field("original",pa.string()))

not_parsed_schema = not_parsed_schema.append(pa.field("good_parse_error",pa.string()))
not_parsed_schema = not_parsed_schema.append(pa.field("good_file_parse_error_start_context",pa.string()))
not_parsed_schema = not_parsed_schema.append(pa.field("good_file_parse_error_highlight",pa.string()))

not_parsed_new = dd.read_parquet('../out_new/sqlglot_unparsed_stmts_only_good_errors/', columns = not_parsed_schema.names ,schema=not_parsed_schema, split_row_groups=True, calculate_divisions=True,engine='pyarrow')

In [57]:
not_parsed_new[not_parsed_new.file_parse_error_highlight != not_parsed_new.good_file_parse_error_highlight]['file_id'].count().compute(scheduler='processes')

[########################################] | 100% Completed | 3.51 ss


51043

In [60]:
not_parsed_new = not_parsed_new[['file_id','statement_id',
                                 'good_parse_error','good_file_parse_error_start_context','good_file_parse_error_highlight',
                                 'original']]

In [62]:
errcounts = not_parsed_new['good_file_parse_error_highlight'].value_counts().compute(scheduler='processes')
err_df = pd.DataFrame({'err':errcounts.index, 'cnt':errcounts.values})
err_df.head(30)

[########################################] | 100% Completed | 11.96 s


,err,cnt
0,DEFAULT|,631853
1,unsigned|,407437
2,UNSIGNED|,194713
3,.|,148417
4,KEY|,116948
5,,112800
6,ON|UPDATE|,59509
7,#|,54997
8,ASC|,48617
9,USING|)|,43710


### DEFAULT

In [69]:
default_check = not_parsed_new[
    (not_parsed_new.good_file_parse_error_highlight.str.contains('DEFAULT',case=False,regex=False)) &
    (
        (not_parsed_new.original.str.contains('`',case=False,regex=False)) |
        (not_parsed_new.original.str.contains('ENGINE',case=False,regex=False))
    ) &
    (
        (not_parsed_new.original.str.contains('CHARSET',case=False,regex=False)) |
        (not_parsed_new.original.str.contains('CHARACTER SET',case=False,regex=False))
    )
]
print(get_stmt_and_file_count(default_check))

[########################################] | 100% Completed | 16.15 s
[########################################] | 100% Completed | 19.06 s
(637179, 91232)


In [67]:
default_check.head()

[########################################] | 100% Completed | 249.81 ms


,file_id,statement_id,good_parse_error,good_file_parse_error_start_context,good_file_parse_error_highlight,original
15,419758,9,Invalid expression / Unexpected token|,"T,\n `name` varchar(20) DEFAULT NULL,\n PRIMARY KEY (`idcategory`)\n) ENGINE=InnoDB AUTO_INCREMENT=7 |",DEFAULT|,"CREATE TABLE IF NOT EXISTS `category` (\n `idcategory` int(11) NOT NULL AUTO_INCREMENT,\n `name` varchar(20) DEFAULT NULL,\n PRIMARY KEY (`idcategory`)\n) ENGINE=InnoDB AUTO_INCREMENT=7 DEFAULT CHARSET=latin1;"
17,419758,15,Invalid expression / Unexpected token|,"LL,\n `pass` varchar(50) DEFAULT NULL,\n PRIMARY KEY (`idpeople`)\n) ENGINE=InnoDB AUTO_INCREMENT=10 |",DEFAULT|,"CREATE TABLE IF NOT EXISTS `people` (\n `idpeople` int(11) NOT NULL AUTO_INCREMENT,\n `name` varchar(20) DEFAULT NULL,\n `email` varchar(50) DEFAULT NULL,\n `pass` varchar(50) DEFAULT NULL,\n PRIMARY KEY (`idpeople`)\n) ENGINE=InnoDB AUTO_INCREMENT=10 DEFAULT CHARSET=latin1;"
19,419764,0,Invalid expression / Unexpected token|,"`key` varchar(255) NOT NULL,\n `value` varchar(255) NOT NULL,\n PRIMARY KEY (`key`)\n) ENGINE=InnoDB |",DEFAULT|,"-- >> This file is autogenerated from tables.sql and constraints.sql. Do not modify directly << --\n--\n-- Table structure for table `ospos_app_config`\n--\n\nCREATE TABLE `ospos_app_config` (\n `key` varchar(255) NOT NULL,\n `value` varchar(255) NOT NULL,\n PRIMARY KEY (`key`)\n) ENGINE=InnoDB DEFAULT CHARSET=utf8;"
25,419764,8,Invalid expression / Unexpected token|,"ULL,\n `percent` decimal(15,2) NOT NULL,\n PRIMARY KEY (`item_id`,`name`,`percent`)\n) ENGINE=InnoDB |",DEFAULT|,"--\n-- Dumping data for table `ospos_items`\n--\n\n\n-- --------------------------------------------------------\n\n--\n-- Table structure for table `ospos_items_taxes`\n--\n\nCREATE TABLE `ospos_items_taxes` (\n `item_id` int(10) NOT NULL,\n `name` varchar(255) NOT NULL,\n `percent` decimal(15,2) NOT NULL,\n PRIMARY KEY (`item_id`,`name`,`percent`)\n) ENGINE=InnoDB DEFAULT CHARSET=utf8;"
26,419764,9,Invalid expression / Unexpected token|,"255) NOT NULL,\n `description` varchar(255) NOT NULL,\n PRIMARY KEY (`item_kit_id`)\n) ENGINE=InnoDB |",DEFAULT|,"--\n-- Dumping data for table `ospos_items_taxes`\n--\n\n\n-- --------------------------------------------------------\n\n--\n-- Table structure for table `ospos_item_kits`\n--\n\nCREATE TABLE `ospos_item_kits` (\n `item_kit_id` int(11) NOT NULL AUTO_INCREMENT,\n `name` varchar(255) NOT NULL,\n `description` varchar(255) NOT NULL,\n PRIMARY KEY (`item_kit_id`)\n) ENGINE=InnoDB DEFAULT CHARSET=utf8 ;"


### UNSIGNED

In [76]:
no_default = not_parsed_new[
    ~((not_parsed_new.good_file_parse_error_highlight.str.contains('DEFAULT',case=False,regex=False)) &
    (
        (not_parsed_new.original.str.contains('`',case=False,regex=False)) |
        (not_parsed_new.original.str.contains('ENGINE',case=False,regex=False))
    ) &
    (
        (not_parsed_new.original.str.contains('CHARSET',case=False,regex=False)) |
        (not_parsed_new.original.str.contains('CHARACTER SET',case=False,regex=False))
    ))
]

unsigned_check = no_default[
    (no_default.good_file_parse_error_highlight.str.contains('UNSIGNED',case=False,regex=False)) &
    (
        (no_default.original.str.contains('INT',case=False,regex=False)) |
        (no_default.original.str.contains('INTEGER',case=False,regex=False)) 
    )
]

print(get_stmt_and_file_count(unsigned_check))

[########################################] | 100% Completed | 19.78 s
[########################################] | 100% Completed | 28.99 s
(602039, 54189)


In [87]:
unsigned_check.sample(frac=0.1).head()

[########################################] | 100% Completed | 773.83 ms


,file_id,statement_id,good_parse_error,good_file_parse_error_start_context,good_file_parse_error_highlight,original
7966,421414,306,Expecting )|Invalid expression / Unexpected token|,CREATE TABLE `reob7_finder_taxonomy` (\n `id` int(10) |,unsigned|,"CREATE TABLE `reob7_finder_taxonomy` (\n `id` int(10) unsigned NOT NULL AUTO_INCREMENT,\n `parent_id` int(10) unsigned NOT NULL DEFAULT '0',\n `title` varchar(255) NOT NULL,\n `state` tinyint(1) unsigned NOT NULL DEFAULT '1',\n `access` tinyint(1) unsigned NOT NULL DEFAULT '0',\n `ordering` tinyint(1) unsigned NOT NULL DEFAULT '0',\n PRIMARY KEY (`id`),\n KEY `parent_id` (`parent_id`),\n KEY `state` (`state`),\n KEY `ordering` (`ordering`),\n KEY `access` (`access`),\n KEY `idx_parent_published` (`parent_id`,`state`,`access`)\n) ENGINE=InnoDB AUTO_INCREMENT=2 DEFAULT CHARSET=utf8;"
15299,422853,8,Expecting )|Invalid expression / Unexpected token|,ble structure for table `activations`\n--\n\nCREATE TABLE IF NOT EXISTS `activations` (\n `id` int(10) |,unsigned|,"-- --------------------------------------------------------\n\n--\n-- Table structure for table `activations`\n--\n\nCREATE TABLE IF NOT EXISTS `activations` (\n `id` int(10) unsigned NOT NULL AUTO_INCREMENT,\n `user_id` int(11) NOT NULL,\n `code` varchar(255) COLLATE utf8_unicode_ci NOT NULL,\n `completed` tinyint(1) NOT NULL DEFAULT '0',\n `completed_at` timestamp NULL DEFAULT NULL,\n `created_at` timestamp NOT NULL DEFAULT '0000-00-00 00:00:00',\n `updated_at` timestamp NOT NULL DEFAULT '0000-00-00 00:00:00',\n PRIMARY KEY (`id`)\n) ENGINE=InnoDB DEFAULT CHARSET=utf8 COLLATE=utf8_unicode_ci AUTO_INCREMENT=3 ;"
23758,424866,155,Expecting )|Invalid expression / Unexpected token|,"LE `posting_journal` (\n `uuid` BINARY(16) NOT NULL,\n `project_id` SMALLINT(5) |",UNSIGNED|,"CREATE TABLE `posting_journal` (\n `uuid` BINARY(16) NOT NULL,\n `project_id` SMALLINT(5) UNSIGNED NOT NULL,\n `fiscal_year_id` MEDIUMINT(8) UNSIGNED DEFAULT NULL,\n `period_id` MEDIUMINT(8) UNSIGNED DEFAULT NULL,\n `trans_id` VARCHAR(100) NOT NULL,\n `trans_date` DATETIME NOT NULL,\n `record_uuid` BINARY(16) NOT NULL, -- previously doc_num\n `description` TEXT,\n `account_id` INT(10) UNSIGNED NOT NULL,\n `debit` DECIMAL(19,4) NOT NULL DEFAULT 0.00,\n `credit` DECIMAL(19,4) NOT NULL DEFAULT 0.00,\n `debit_equiv` DECIMAL(19,4) NOT NULL DEFAULT 0.00,\n `credit_equiv` DECIMAL(19,4) NOT NULL DEFAULT 0.00,\n `currency_id` TINYINT(3) UNSIGNED NOT NULL,\n `entity_uuid` BINARY(16), -- previously deb_cred_uuid\n `reference_uuid` BINARY(16), -- previously inv_po_id\n `comment` TEXT,\n `origin_id` TINYINT(3) UNSIGNED NULL,\n `user_id` SMALLINT(5) UNSIGNED NOT NULL,\n `cc_id` SMALLINT(6),\n `pc_id` SMALLINT(6),\n PRIMARY KEY (`uuid`),\n KEY `project_id` (`project_id`),\n KEY `fiscal_year_id` (`fiscal_year_id`),\n KEY `period_id` (`period_id`),\n KEY `currency_id` (`currency_id`),\n KEY `user_id` (`user_id`),\n KEY `cc_id` (`cc_id`),\n KEY `pc_id` (`pc_id`),\n INDEX `trans_date` (`trans_date`),\n INDEX `trans_id` (`trans_id`),\n INDEX `record_uuid` (`record_uuid`),\n INDEX `reference_uuid` (`record_uuid`),\n INDEX `entity_uuid` (`entity_uuid`),\n INDEX `account_id` (`account_id`),\n FOREIGN KEY (`fiscal_year_id`) REFERENCES `fiscal_year` (`id`),\n FOREIGN KEY (`period_id`) REFERENCES `period` (`id`),\n FOREIGN KEY (`project_id`) REFERENCES `project` (`id`) ON UPDATE CASCADE,\n FOREIGN KEY (`account_id`) REFERENCES `account` (`id`),\n FOREIGN KEY (`currency_id`) REFERENCES `currency` (`id`) ON UPDATE CASCADE,\n FOREIGN KEY (`user_id`) REFERENCES `user` (`id`) ON UPDATE CASCADE,\n FOREIGN KEY (`cc_id`) REFERENCES `cost_center` (`id`) ON UPDATE CASCADE,\n FOREIGN KEY (`pc_id`) REFERENCES `profit_center` (`id`) ON UPDATE CASCADE\n) ENGINE=InnoDB DEFAULT CHARSET=utf8;"
3565,420548,29,Expecting )|Invalid expression / Unexpected token|,able structure for table `permission_role`\n--\n\nCREATE TABLE `permission_role` (\n `role_id` int(10) |,UNSIGNED|,"-- ------------------------

### dot

In [95]:
no_unsigned = no_default[
    ~((no_default.good_file_parse_error_highlight.str.contains('UNSIGNED',case=False,regex=False)) &
    (
        (no_default.original.str.contains('INT',case=False,regex=False)) |
        (no_default.original.str.contains('INTEGER',case=False,regex=False)) 
    ))
]
dot_check = no_unsigned[
    (no_unsigned.good_file_parse_error_highlight == '.|') &
    ((no_unsigned.good_file_parse_error_start_context.str.contains('references',case=False,regex=False)) |
    (no_unsigned.good_file_parse_error_start_context.str.contains('index',case=False,regex=False)))
]
print(get_stmt_and_file_count(dot_check))
#dot_check.sample(frac=0.1).head()
# 150k
# 65k for references
# 76k for index

[########################################] | 100% Completed | 20.07 s
[########################################] | 100% Completed | 25.31 s
(141601, 14111)


In [93]:
dot_check.sample(frac=0.1).head()

[########################################] | 100% Completed | 718.13 ms


,file_id,statement_id,good_parse_error,good_file_parse_error_start_context,good_file_parse_error_highlight,original
7379,421290,212,Invalid expression / Unexpected token|,CREATE INDEX IF NOT EXISTS T_ILI2DB_IMPORT_BASKET_importrun_idx ON stage|,.|,CREATE INDEX IF NOT EXISTS T_ILI2DB_IMPORT_BASKET_importrun_idx ON stage.t_ili2db_import_basket ( importrun );
25934,425193,93,Required keyword: 'this' missing for <class 'sqlglot.expressions.Table'>|Invalid expression / Unexpected token|,"R_ID_C564EBA6\n--------------------------------------------------------\n\n CREATE INDEX ""C##DBETAPA1""|",.|,"--------------------------------------------------------\n-- DDL for Index DJANGO_ADM_USER_ID_C564EBA6\n--------------------------------------------------------\n\n CREATE INDEX ""C##DBETAPA1"".""DJANGO_ADM_USER_ID_C564EBA6"" ON ""C##DBETAPA1"".""DJANGO_ADMIN_LOG"" (""USER_ID"") \n PCTFREE 10 INITRANS 2 MAXTRANS 255 COMPUTE STATISTICS \n STORAGE(INITIAL 65536 NEXT 1048576 MINEXTENTS 1 MAXEXTENTS 2147483645\n PCTINCREASE 0 FREELISTS 1 FREELIST GROUPS 1\n BUFFER_POOL DEFAULT FLASH_CACHE DEFAULT CELL_FLASH_CACHE DEFAULT)\n TABLESPACE ""USERS"" ;"
7567,421323,3,Invalid expression / Unexpected token|,|,.|,.\nCOMMENT ON COLUMN oe.customers.cust_geo_location IS\n'SDO (spatial) column.';
16814,423250,8,Invalid expression / Unexpected token|,create index cities_byname on geo|,.|,create index cities_byname on geo.cities(name);
17859,423510,13,Invalid expression / Unexpected token|,CREATE INDEX idx_und_fdrl_mun ON public|,.|,CREATE INDEX idx_und_fdrl_mun ON public.mun USING btree (nr_und_fdrl);


### KEY

In [104]:
no_dot = no_unsigned[
    ~((no_unsigned.good_file_parse_error_highlight == '.|') &
    ((no_unsigned.good_file_parse_error_start_context.str.contains('references',case=False,regex=False)) |
    (no_unsigned.good_file_parse_error_start_context.str.contains('index',case=False,regex=False))))
]
key_check = no_dot[
    (no_dot.good_file_parse_error_highlight.str.upper() == 'KEY|') &
    (no_dot.good_file_parse_error_start_context.str.contains('unique',case=False,regex=False)) &
    (
        (no_dot.original.str.contains('`',case=False,regex=False)) |
        (no_dot.original.str.contains('ENGINE',case=False,regex=False))
    )
]
print(get_stmt_and_file_count(key_check))
#dot_check.sample(frac=0.1).head()

[########################################] | 100% Completed | 25.37 s
[########################################] | 100% Completed | 27.46 s
(114168, 27731)


In [100]:
key_check.sample(frac=0.1).head()

[########################################] | 100% Completed | 793.45 ms


,file_id,statement_id,good_parse_error,good_file_parse_error_start_context,good_file_parse_error_highlight,original
4135,420645,322,Expecting )|Invalid expression / Unexpected token|,",\n ordernumber_to int(11) NULL,\n timestamp timestamp NOT NULL DEFAULT CURRENT_TIMESTAMP,\n UNIQUE |",KEY|,"CREATE TABLE aqorders_transfers (\n ordernumber_from int(11) NULL,\n ordernumber_to int(11) NULL,\n timestamp timestamp NOT NULL DEFAULT CURRENT_TIMESTAMP,\n UNIQUE KEY ordernumber_from (ordernumber_from),\n UNIQUE KEY ordernumber_to (ordernumber_to),\n CONSTRAINT aqorders_transfers_ordernumber_from FOREIGN KEY (ordernumber_from) REFERENCES aqorders (ordernumber) ON DELETE SET NULL ON UPDATE CASCADE,\n CONSTRAINT aqorders_transfers_ordernumber_to FOREIGN KEY (ordernumber_to) REFERENCES aqorders (ordernumber) ON DELETE SET NULL ON UPDATE CASCADE\n) ENGINE=InnoDB DEFAULT CHARSET=utf8 COLLATE=utf8_unicode_ci;"
6627,421196,50,Expecting )|Invalid expression / Unexpected token|,"L,\n `shelf_id_id` int NOT NULL,\n `category` varchar(255) NOT NULL,\n PRIMARY KEY (`id`),\n UNIQUE |",KEY|,"CREATE TABLE `books_books` (\n `id` int NOT NULL AUTO_INCREMENT,\n `isbn_number` varchar(45) NOT NULL,\n `copy_number` int NOT NULL,\n `title` varchar(255) NOT NULL,\n `author` varchar(255) NOT NULL,\n `year_of_publication` int NOT NULL,\n `current_status` varchar(255) NOT NULL,\n `Img1` varchar(100) NOT NULL,\n `shelf_id_id` int NOT NULL,\n `category` varchar(255) NOT NULL,\n PRIMARY KEY (`id`),\n UNIQUE KEY `books_books_isbn_number_copy_number_17539bff_uniq` (`isbn_number`,`copy_number`),\n KEY `books_books_shelf_id_id_a491fcdd_fk_books_shelf_shelf_id` (`shelf_id_id`),\n CONSTRAINT `books_books_shelf_id_id_a491fcdd_fk_books_shelf_shelf_id` FOREIGN KEY (`shelf_id_id`) REFERENCES `books_shelf` (`shelf_id`)\n) ENGINE=InnoDB AUTO_INCREMENT=7 DEFAULT CHARSET=utf8mb4 COLLATE=utf8mb4_0900_ai_ci;"
7043,421268,210,Expecting )|Invalid expression / Unexpected token|,"otionCommentId` int(11) NOT NULL,\n `likes` tinyint(4) DEFAULT NULL,\n PRIMARY KEY (`id`),\n UNIQUE |",KEY|,"CREATE TABLE `motionCommentSupporter` (\n `id` int(11) NOT NULL AUTO_INCREMENT,\n `ipHash` char(32) DEFAULT NULL,\n `cookieId` int(11) DEFAULT NULL,\n `motionCommentId` int(11) NOT NULL,\n `likes` tinyint(4) DEFAULT NULL,\n PRIMARY KEY (`id`),\n UNIQUE KEY `ip_hash_motion` (`ipHash`,`motionCommentId`),\n UNIQUE KEY `cookie_motion` (`cookieId`,`motionCommentId`),\n KEY `fk_motion_comment_supporter_commentIdx` (`motionCommentId`),\n CONSTRAINT `fk_motion_comment_supporter_comment` FOREIGN KEY (`motionCommentId`) REFERENCES `motionComment` (`id`) ON DELETE NO ACTION ON UPDATE NO ACTION\n) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4;"
8156,421418,551,Expecting )|Invalid expression / Unexpected token|,"NT 'auto fill server date and time zone',\n PRIMARY KEY (`hethi_service_id`,`service_id`),\n UNIQUE |",KEY|,"CREATE TABLE `chethi_services` (\n `hethi_service_id` char(10) NOT NULL COMMENT 'total 6 characters. Starts with ch, 100101. example : ch100101',\n `service_id` char(10) NOT NULL COMMENT 'FK. cCore_services.service_id',\n `hethi_servicecode` varchar(10) NOT NULL COMMENT 'DaaS, BaaS, OaaS, IaaS, AaaS,StaaS',\n `lastupuser` char(6) NOT NULL COMMENT 'user ID audit trail',\n `lastupdtm` datetime NOT NULL COMMENT 'auto fill server date and time zone',\n PRIMARY KEY (`hethi_service_id`,`service_id`),\n UNIQUE KEY `hethi_servicecode_UNIQUE` (`hethi_servicecode`),\n KEY `service_id_idx` (`service_id`),\n CONSTRAINT `service_id` FOREIGN KEY (`service_id`) REFERENCES `ccore_services` (`service_id`) ON DELETE NO ACTION ON UPDATE NO ACTION\n) ENGINE=InnoDB DEFAULT CHARSET=utf8;"
14133,422677,83,Expecting )|Invalid expression / Unexpected token|,"NOT NULL,\n `phoneNumber` varchar(15) DEFAULT NULL,\n PRIMARY KEY (`schoolDepartmentID`),\n UNIQUE |",KEY|,"CREATE TABLE `schools_departments` (\n `schoolDepartmentID` int(11) NOT NULL AUTO_INCREMENT,\n `name` varchar(64) NOT NULL,\n `abbreviation` varchar(3) DEFAULT NULL,\n `div

### empty

In [110]:
no_key = no_dot[
    ~((no_dot.good_file_parse_error_highlight.str.upper() == 'KEY|') &
    (no_dot.good_file_parse_error_start_context.str.contains('unique',case=False,regex=False)) &
    (
        (no_dot.original.str.contains('`',case=False,regex=False)) |
        (no_dot.original.str.contains('ENGINE',case=False,regex=False))
    ))
]
empty_check = no_key[
    (no_key.good_file_parse_error_highlight == '') &
    (no_key.good_parse_error.str.contains('NON_PARSER_ERROR|',case=False,regex=False))
]

empty_alter_check = no_key[
    (no_key.good_file_parse_error_highlight == '') &
    (no_key.good_parse_error.str.contains('NON_PARSER_ERROR|',case=False,regex=False)) &
    (no_key.original.str.contains('alter table',case=False,regex=False))
]

empty_insert_into_check = no_key[
    (no_key.good_file_parse_error_highlight == '') &
    (no_key.good_parse_error.str.contains('NON_PARSER_ERROR|',case=False,regex=False)) &
    (no_key.original.str.contains('insert into',case=False,regex=False))
]
print(get_stmt_and_file_count(empty_insert_into_check))
#dot_check.sample(frac=0.1).head()

[########################################] | 100% Completed | 28.62 s
[########################################] | 100% Completed | 38.47 s
(51259, 16829)


In [106]:
empty_check.sample(frac=0.1).head()

[########################################] | 100% Completed | 1.03 ss


,file_id,statement_id,good_parse_error,good_file_parse_error_start_context,good_file_parse_error_highlight,original
26063,425232,6,NON_PARSER_ERROR|local variable 'expression' referenced before assignment,,,alter table Department\nadd constraint foreign key fk_department_manager(manager_id) references Employee(manager_id) on delete no action;
13494,422636,201,NON_PARSER_ERROR|local variable 'expression' referenced before assignment,,,alter table DM_DIMORGANISATION\n add constraint AK_KEY_2_DM_DIMOR unique (ORGNAME);
28688,425803,292,NON_PARSER_ERROR|Missing ' from 1:102,,,"INSERT INTO `points_of_interest` VALUES ('290', '-1239.75', '104.888', '7', '99', '0', 'Karn\'s Smithy');"
24639,425052,174,"NON_PARSER_ERROR|Missing "" from 4:135",,,"}\n\tif(!isnull(column_where2) && !isnull(row_identifier3) && row_identifier2 == -1)\n\t{\n\t\tformat(SQL::upd_form2, sizeof(SQL::upd_form2), "" AND `%s`='%s'"", column_where2, row_identifier3);"
20591,424209,296,NON_PARSER_ERROR|Missing ' from 1:318,,,"INSERT INTO `catalog_pages` VALUES ('25833', '25820', 'Triangular Prism', '1', '193', '1', '1', '1', '0', '75', 'default_3x3', 'Builders_Club_Header', 'Builders_Club_Teaser', ' ', 'In this section, you\'ll find over 100 building blocks avaible to you in order to create your perfect room!', ' ', ' ', ' ', '0', ' ', ' ');"


### ON|UPDATE|

In [113]:
no_empty = no_key[
    ~((no_key.good_file_parse_error_highlight == '') &
    (no_key.good_parse_error.str.contains('NON_PARSER_ERROR|',case=False,regex=False)))
]

on_update_check = no_empty[
    (no_empty.good_file_parse_error_highlight == 'ON|UPDATE|') &
    (no_empty.good_file_parse_error_start_context.str.contains('timestamp',case=False,regex=False))
]
print(get_stmt_and_file_count(on_update_check))
#dot_check.sample(frac=0.1).head()

[########################################] | 100% Completed | 28.05 s
[########################################] | 100% Completed | 31.86 s
(57091, 13226)


In [112]:
on_update_check.sample(frac=0.5).head()

[########################################] | 100% Completed | 1.15 ss


c:\Users\geacu\anaconda3\envs\windows\lib\site-packages\dask\core.py:119: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  return func(*(_execute_task(a, cache) for a in args))


,file_id,statement_id,good_parse_error,good_file_parse_error_start_context,good_file_parse_error_highlight,original
6102,421114,130,Expecting )|,"MENT,\n `title` varchar(255) DEFAULT NULL,\n `create_date` timestamp NULL DEFAULT CURRENT_TIMESTAMP |T,\n `title` varchar(255) DEFAULT NULL,\n `create_date` timestamp NULL DEFAULT CURRENT_TIMESTAMP ON |",ON|UPDATE|,"CREATE TABLE `post` (\n `id` int(11) NOT NULL AUTO_INCREMENT,\n `title` varchar(255) DEFAULT NULL,\n `create_date` timestamp NULL DEFAULT CURRENT_TIMESTAMP ON UPDATE CURRENT_TIMESTAMP,\n `type` tinyint(4) DEFAULT NULL,\n `content` text,\n `image` varchar(255) DEFAULT NULL,\n `status` tinyint(4) DEFAULT NULL,\n PRIMARY KEY (`id`)\n) ENGINE=InnoDB DEFAULT CHARSET=utf8;"
16230,423094,18,Expecting )|,"(12) NOT NULL,\n `phone` char(12) DEFAULT NULL,\n `tm` timestamp NOT NULL DEFAULT CURRENT_TIMESTAMP |) NOT NULL,\n `phone` char(12) DEFAULT NULL,\n `tm` timestamp NOT NULL DEFAULT CURRENT_TIMESTAMP ON |",ON|UPDATE|,"CREATE TABLE `Log` (\n `IP` char(16) DEFAULT NULL,\n `MAC` char(12) NOT NULL,\n `phone` char(12) DEFAULT NULL,\n `tm` timestamp NOT NULL DEFAULT CURRENT_TIMESTAMP ON UPDATE CURRENT_TIMESTAMP,\n `msg` varchar(100) DEFAULT NULL\n) ENGINE=MyISAM DEFAULT CHARSET=latin1;"
10644,421971,43,Expecting )|,"ULL,\n `jabatan_baru` int(3) NOT NULL,\n `timestamp` timestamp NOT NULL DEFAULT current_timestamp() |,\n `jabatan_baru` int(3) NOT NULL,\n `timestamp` timestamp NOT NULL DEFAULT current_timestamp() ON |",ON|UPDATE|,"CREATE TABLE `tbl_log_jabatan` (\n `id_log_jabatan` int(11) NOT NULL AUTO_INCREMENT,\n `id_pegawai` int(3) NOT NULL,\n `jabatan_lama` int(3) NOT NULL,\n `jabatan_baru` int(3) NOT NULL,\n `timestamp` timestamp NOT NULL DEFAULT current_timestamp() ON UPDATE current_timestamp(),\n PRIMARY KEY (`id_log_jabatan`),\n KEY `pegawai` (`id_pegawai`),\n KEY `jabatan_lama` (`jabatan_lama`),\n KEY `jabatan_baru` (`jabatan_baru`),\n CONSTRAINT `jabatan_baru` FOREIGN KEY (`jabatan_baru`) REFERENCES `tbl_jabatan` (`id_jabatan`),\n CONSTRAINT `jabatan_lama` FOREIGN KEY (`jabatan_lama`) REFERENCES `tbl_jabatan` (`id_jabatan`),\n CONSTRAINT `pegawai` FOREIGN KEY (`id_pegawai`) REFERENCES `tbl_pegawai` (`id_pegawai`)\n) ENGINE=InnoDB AUTO_INCREMENT=3 DEFAULT CHARSET=utf8mb4;"
18868,423778,113,Expecting )|,"atus` varchar(255) COLLATE utf8_unicode_ci DEFAULT NULL,\n `created_at` timestamp NULL DEFAULT NULL |s` varchar(255) COLLATE utf8_unicode_ci DEFAULT NULL,\n `created_at` timestamp NULL DEFAULT NULL ON |",ON|UPDATE|,"CREATE TABLE `reasons` (\n `id` int(11) NOT NULL AUTO_INCREMENT,\n `title` varchar(255) COLLATE utf8_unicode_ci DEFAULT NULL,\n `caption` text COLLATE utf8_unicode_ci,\n `image` int(11) DEFAULT NULL,\n `status` varchar(255) COLLATE utf8_unicode_ci DEFAULT NULL,\n `created_at` timestamp NULL DEFAULT NULL ON UPDATE CURRENT_TIMESTAMP,\n `updated_at` timestamp NULL DEFAULT NULL ON UPDATE CURRENT_TIMESTAMP,\n `link` text COLLATE utf8_unicode_ci,\n `order` int(11) DEFAULT NULL,\n PRIMARY KEY (`id`)\n) ENGINE=InnoDB AUTO_INCREMENT=12 DEFAULT CHARSET=utf8 COLLATE=utf8_unicode_ci;"
16625,423208,91,Expecting )|,"R SET latin1 COLLATE latin1_swedish_ci NOT NULL DEFAULT 'System',\n `updated` timestamp(0) NOT NULL |ET latin1 COLLATE latin1_swedish_ci NOT NULL DEFAULT 'System',\n `updated` timestamp(0) NOT NULL ON |",ON|UPDATE|,"CREATE TABLE `hot_extra` (\n `extra_id` int(11) NOT NULL AUTO_INCREMENT,\n `extra_name` varchar(128) CHARACTER SET latin1 COLLATE latin1_swedish_ci NOT NULL,\n `extra_charge` float(10, 2) NOT NULL,\n `created` timestamp(0) NOT NULL DEFAULT CURRENT_TIMESTAMP,\n `created_by` varchar(50) CHARACTER SET latin1 COLLATE latin1_swedish_ci NOT NULL DEFAULT 'System',\n `updated` timestamp(0) NOT NULL ON UPDATE CURRENT_TIMESTAMP(0),\n `updated_by` varchar(20) CHARACTER SET latin1 COLLATE latin1_swedish_ci NOT NULL DEFAULT 'System',\n `is_active` tinyint(1) NOT NULL DEFAULT 1,\n `is_deleted` tinyint(1) NOT NULL DEFAULT 0,\n PRIMARY KEY (`extra_id`) USING BTREE\n) ENGINE 

### writing down again smaller subset of errs and recalculating errors for hashtag

In [140]:
no_on_pdate = no_empty[
   ~((no_empty.good_file_parse_error_highlight == 'ON|UPDATE|') &
    (no_empty.good_file_parse_error_start_context.str.contains('timestamp',case=False,regex=False))
   )
]
#print(get_stmt_and_file_count(no_on_pdate))
#dot_check.sample(frac=0.1).head()

In [141]:
import sqlglot
def get_good2_error_1(row):
    parsed_statements = None
    _dialect = None
    if row['good_file_parse_error_highlight'] == '#|':
        _dialect = "mysql"

    if "TIMEOUT_ERROR" in row['good_parse_error']:
        _dialect = None

    if _dialect is not None:
        _result = {}
        try:
            parsed_statements = sqlglot.parse(row['original'],read=_dialect,error_level=sqlglot.ErrorLevel.RAISE)
        except sqlglot.errors.ParseError as e:
            _result["parse_error"] = ''
            _result["file_parse_error_start_context"] = ''
            _result["file_parse_error_highlight"] = ''
            for idx, err in enumerate(e.errors):
                if idx > 0:
                    if err['highlight'] != e.errors[idx-1]['highlight']:
                        _result["file_parse_error_start_context"] += err['start_context'][0:200] + '|'
                        _result["file_parse_error_highlight"] += err['highlight'][0:200] + '|'
                    else:
                        _result["parse_error"] += err['description'][0:200] + '|'
                else:
                    _result["parse_error"] += err['description'][0:200] + '|'
                    
                    _result["file_parse_error_start_context"] += err['start_context'][0:200] + '|'
                    _result["file_parse_error_highlight"] += err['highlight'][0:200] + '|'

            _result["parse_error"] = _result["parse_error"][0:600]
            _result["file_parse_error_start_context"] = _result["file_parse_error_start_context"][0:400]
            _result["file_parse_error_highlight"] = _result["file_parse_error_highlight"][0:200]
        except Exception as e:
            _result['parse_error'] = 'NON_PARSER_ERROR|' + str(e)[0:300]
            _result['file_parse_error_start_context'] = ''
            _result['file_parse_error_highlight'] = ''
    else:
        _result = {}
        _result["parse_error"] = row['good_parse_error']
        _result["file_parse_error_start_context"] = row['good_file_parse_error_start_context']
        _result["file_parse_error_highlight"] = row['good_file_parse_error_highlight']
    
    return _result["parse_error"]

def get_good2_error_2(row):
    parsed_statements = None
    _dialect = None
    if row['good_file_parse_error_highlight'] == '#|':
        _dialect = "mysql"

    if "TIMEOUT_ERROR" in row['good_parse_error']:
        _dialect = None

    if _dialect is not None:
        _result = {}
        try:
            parsed_statements = sqlglot.parse(row['original'],read=_dialect,error_level=sqlglot.ErrorLevel.RAISE)
        except sqlglot.errors.ParseError as e:
            _result["parse_error"] = ''
            _result["file_parse_error_start_context"] = ''
            _result["file_parse_error_highlight"] = ''
            for idx, err in enumerate(e.errors):
                if idx > 0:
                    if err['highlight'] != e.errors[idx-1]['highlight']:
                        _result["file_parse_error_start_context"] += err['start_context'][0:200] + '|'
                        _result["file_parse_error_highlight"] += err['highlight'][0:200] + '|'
                    else:
                        _result["parse_error"] += err['description'][0:200] + '|'
                else:
                    _result["parse_error"] += err['description'][0:200] + '|'
                    
                    _result["file_parse_error_start_context"] += err['start_context'][0:200] + '|'
                    _result["file_parse_error_highlight"] += err['highlight'][0:200] + '|'

            _result["parse_error"] = _result["parse_error"][0:600]
            _result["file_parse_error_start_context"] = _result["file_parse_error_start_context"][0:400]
            _result["file_parse_error_highlight"] = _result["file_parse_error_highlight"][0:200]
        except Exception as e:
            _result['parse_error'] = 'NON_PARSER_ERROR|' + str(e)[0:300]
            _result['file_parse_error_start_context'] = ''
            _result['file_parse_error_highlight'] = ''
    else:
        _result = {}
        _result["parse_error"] = row['good_parse_error']
        _result["file_parse_error_start_context"] = row['good_file_parse_error_start_context']
        _result["file_parse_error_highlight"] = row['good_file_parse_error_highlight']
    
    return _result["file_parse_error_start_context"]

def get_good2_error_3(row):
    parsed_statements = None
    _dialect = None
    if row['good_file_parse_error_highlight'] == '#|':
        _dialect = "mysql"

    if "TIMEOUT_ERROR" in row['good_parse_error']:
        _dialect = None

    if _dialect is not None:
        _result = {}
        try:
            parsed_statements = sqlglot.parse(row['original'],read=_dialect,error_level=sqlglot.ErrorLevel.RAISE)
        except sqlglot.errors.ParseError as e:
            _result["parse_error"] = ''
            _result["file_parse_error_start_context"] = ''
            _result["file_parse_error_highlight"] = ''
            for idx, err in enumerate(e.errors):
                if idx > 0:
                    if err['highlight'] != e.errors[idx-1]['highlight']:
                        _result["file_parse_error_start_context"] += err['start_context'][0:200] + '|'
                        _result["file_parse_error_highlight"] += err['highlight'][0:200] + '|'
                    else:
                        _result["parse_error"] += err['description'][0:200] + '|'
                else:
                    _result["parse_error"] += err['description'][0:200] + '|'
                    
                    _result["file_parse_error_start_context"] += err['start_context'][0:200] + '|'
                    _result["file_parse_error_highlight"] += err['highlight'][0:200] + '|'

            _result["parse_error"] = _result["parse_error"][0:600]
            _result["file_parse_error_start_context"] = _result["file_parse_error_start_context"][0:400]
            _result["file_parse_error_highlight"] = _result["file_parse_error_highlight"][0:200]
        except Exception as e:
            _result['parse_error'] = 'NON_PARSER_ERROR|' + str(e)[0:300]
            _result['file_parse_error_start_context'] = ''
            _result['file_parse_error_highlight'] = ''
    else:
        _result = {}
        _result["parse_error"] = row['good_parse_error']
        _result["file_parse_error_start_context"] = row['good_file_parse_error_start_context']
        _result["file_parse_error_highlight"] = row['good_file_parse_error_highlight']
    
    return _result["file_parse_error_highlight"]

In [142]:
no_on_pdate['good2_parse_error'] = no_on_pdate.apply(
    get_good2_error_1,
    axis=1,
    meta=('good2_parse_error', 'string')
)

no_on_pdate['good2_file_parse_error_start_context'] = no_on_pdate.apply(
    get_good2_error_2,
    axis=1,
    meta=('good2_file_parse_error_start_context', 'string')
)

no_on_pdate['good2_file_parse_error_highlight'] = no_on_pdate.apply(
    get_good2_error_3,
    axis=1,
    meta=('good2_file_parse_error_highlight', 'string')
)

In [145]:
no_on_pdate = no_on_pdate[['file_id','statement_id','good2_parse_error',
                          'good2_file_parse_error_start_context',
                          'good2_file_parse_error_highlight',
                          'original']]

In [147]:
not_parsed_new_schema = pa.schema([])
not_parsed_new_schema = not_parsed_new_schema.append(pa.field("file_id",pa.string()))
not_parsed_new_schema = not_parsed_new_schema.append(pa.field("statement_id",pa.int64()))
not_parsed_new_schema = not_parsed_new_schema.append(pa.field("good2_parse_error",pa.string()))
not_parsed_new_schema = not_parsed_new_schema.append(pa.field("good2_file_parse_error_start_context",pa.string()))
not_parsed_new_schema = not_parsed_new_schema.append(pa.field("good2_file_parse_error_highlight",pa.string()))
not_parsed_new_schema = not_parsed_new_schema.append(pa.field("original",pa.string()))

In [148]:
#no_on_pdate.to_parquet('../out_new/sqlglot_unparsed_stmts_only_good_errors_smaller_subset/',schema=not_parsed_new_schema,overwrite=True,
compute_kwargs={'scheduler':'processes'})

[########################################] | 100% Completed | 227.06 s


## Reading smaller subset of errors after going through some (from 3.65 mil to 2 mil)

### Read and Prep

In [149]:
not_parsed_new_schema = pa.schema([])
not_parsed_new_schema = not_parsed_new_schema.append(pa.field("file_id",pa.string()))
not_parsed_new_schema = not_parsed_new_schema.append(pa.field("statement_id",pa.int64()))
not_parsed_new_schema = not_parsed_new_schema.append(pa.field("good2_parse_error",pa.string()))
not_parsed_new_schema = not_parsed_new_schema.append(pa.field("good2_file_parse_error_start_context",pa.string()))
not_parsed_new_schema = not_parsed_new_schema.append(pa.field("good2_file_parse_error_highlight",pa.string()))
not_parsed_new_schema = not_parsed_new_schema.append(pa.field("original",pa.string()))
not_parsed_smaller = dd.read_parquet('../out_new/sqlglot_unparsed_stmts_only_good_errors_smaller_subset/', columns = not_parsed_new_schema.names ,schema=not_parsed_new_schema, split_row_groups=True, calculate_divisions=True,engine='pyarrow')

In [150]:
def get_stmt_and_file_count(dask_df):
    x = dask_df['file_id'].nunique().compute(scheduler='processes')
    y = dask_df['file_id'].count().compute(scheduler='processes')
    return y,x

### Err counts

In [217]:
errcounts = no_install_thing['good2_file_parse_error_highlight'].value_counts().compute(scheduler='processes')
err_df = pd.DataFrame({'err':errcounts.index, 'cnt':errcounts.values})
err_df.head(30)

[########################################] | 100% Completed | 23.42 s


,err,cnt
0,INTO|,21149
1,ALTER|,20589
2,\|,19159
3,restrict|,18421
4,CREATE|,15951
5,RESTRICT|,14395
6,OBJECT_ID|,13688
7,CONSTRAINTS|,13435
8,DROP|,13330
9,drop|,13108


### ASC

In [154]:
asc_check = not_parsed_smaller[
   (not_parsed_smaller.good2_file_parse_error_highlight.str.upper() == 'ASC|') &
   ((not_parsed_smaller.original.str.contains('INDEX',case=False,regex=False)) |
   (not_parsed_smaller.original.str.contains('CONSTRAINT',case=False,regex=False)))
]
print(get_stmt_and_file_count(asc_check))
#dot_check.sample(frac=0.6).head()

[########################################] | 100% Completed | 6.93 ss
[########################################] | 100% Completed | 7.15 ss
(49058, 12778)


In [153]:
asc_check.sample(frac=0.6).head()

[########################################] | 100% Completed | 248.14 ms


,file_id,statement_id,good2_parse_error,good2_file_parse_error_start_context,good2_file_parse_error_highlight,original
2140,420653,0,Expecting )|Expecting )|Invalid expression / Unexpected token|,"erID] INT NULL,\n CONSTRAINT [PK_TitleVariant] PRIMARY KEY CLUSTERED ([TitleVariantID] |",ASC|,"CREATE TABLE [dbo].[TitleVariant] (\n [TitleVariantID] INT IDENTITY (1, 1) NOT NULL,\n [TitleID] INT NOT NULL,\n [TitleVariantTypeID] INT NOT NULL,\n [Title] NVARCHAR (MAX) CONSTRAINT [DF_TitleVariant_Title] DEFAULT ('') NOT NULL,\n [TitleRemainder] NVARCHAR (MAX) CONSTRAINT [DF_TitleVariant_TitleRemainder] DEFAULT ('') NOT NULL,\n [PartNumber] NVARCHAR (255) CONSTRAINT [DF_TitleVariant_PartNumber] DEFAULT ('') NOT NULL,\n [PartName] NVARCHAR (255) CONSTRAINT [DF_TitleVariant_PartName] DEFAULT ('') NOT NULL,\n [CreationDate] DATETIME CONSTRAINT [DF_TitleVariant_CreationDate] DEFAULT (getdate()) NOT NULL,\n [LastModifiedDate] DATETIME CONSTRAINT [DF_TitleVariant_LastModifiedDate] DEFAULT (getdate()) NOT NULL,\n [CreationUserID] INT NULL,\n [LastModifiedUserID] INT NULL,\n CONSTRAINT [PK_TitleVariant] PRIMARY KEY CLUSTERED ([TitleVariantID] ASC),\n CONSTRAINT [FK_TitleVariant_Title] FOREIGN KEY ([TitleID]) REFERENCES [dbo].[Title] ([TitleID]),\n CONSTRAINT [FK_TitleVariant_TitleVariantType] FOREIGN KEY ([TitleVariantTypeID]) REFERENCES [dbo].[TitleVariantType] ([TitleVariantTypeID])\n);"
6882,422636,322,Expecting )|Invalid expression / Unexpected token|,================*/\ncreate index RELATIONSHIP_169_FK on DM_FACTCASEPARTICIPANT (\n ACTIVEENDDATEKEY |,ASC|,/*==============================================================*/\n/* Index: RELATIONSHIP_169_FK */\n/*==============================================================*/\ncreate index RELATIONSHIP_169_FK on DM_FACTCASEPARTICIPANT (\n ACTIVEENDDATEKEY ASC\n);
6948,422636,401,Expecting )|Invalid expression / Unexpected token|,===============*/\ncreate index RELATIONSHIP_151_FK on DM_FACTPRODUCTAWARDRECIPIENT (\n AGEGROUPKEY |,ASC|,/*==============================================================*/\n/* Index: RELATIONSHIP_151_FK */\n/*==============================================================*/\ncreate index RELATIONSHIP_151_FK on DM_FACTPRODUCTAWARDRECIPIENT (\n AGEGROUPKEY ASC\n);
14754,425463,38,Expecting )|Invalid expression / Unexpected token|,CREATE INDEX ADT_MGMT_PROV_ID_X ON AUDIT_MGMT_PROVISIONING (EVENT_ID |,ASC|,CREATE INDEX ADT_MGMT_PROV_ID_X ON AUDIT_MGMT_PROVISIONING (EVENT_ID ASC) MINPCTUSED 10 ALLOW REVERSE SCANS;
11809,424486,68,Expecting )|Invalid expression / Unexpected token|,CREATE INDEX idx_sys_company_ts ON sys_company (tree_sort |,ASC|,CREATE INDEX idx_sys_company_ts ON sys_company (tree_sort ASC);


### USING

In [158]:
no_asc = not_parsed_smaller[
   ~((not_parsed_smaller.good2_file_parse_error_highlight.str.upper() == 'ASC|') &
   ((not_parsed_smaller.original.str.contains('INDEX',case=False,regex=False)) |
   (not_parsed_smaller.original.str.contains('CONSTRAINT',case=False,regex=False))))
]

using_check = no_asc[
    (no_asc.good2_file_parse_error_highlight.str.contains('USING',case=False,regex=False)) &
    ((no_asc.original.str.contains('BTREE',case=False,regex=False)) |
    ((no_asc.original.str.contains('INDEX',case=False,regex=False))))
]
print(get_stmt_and_file_count(using_check))
#dot_check.sample(frac=0.6).head()

[########################################] | 100% Completed | 10.10 s
[########################################] | 100% Completed | 10.15 s
(107498, 13044)


In [157]:
using_check.sample(frac=0.6).head()

[########################################] | 100% Completed | 317.77 ms


,file_id,statement_id,good2_parse_error,good2_file_parse_error_start_context,good2_file_parse_error_highlight,original
8785,423134,322,Invalid expression / Unexpected token|,: INDEX; Schema: public; Owner: root\n--\n\nCREATE INDEX django_admin_log_417f1b1c ON django_admin_log |,USING|,--\n-- Name: django_admin_log_417f1b1c; Type: INDEX; Schema: public; Owner: root\n--\n\nCREATE INDEX django_admin_log_417f1b1c ON django_admin_log USING btree (content_type_id);
4276,421495,6,Expecting )|,"TE utf8_general_ci NULL DEFAULT NULL,\n PRIMARY KEY (`SCHED_NAME`, `TRIGGER_NAME`, `TRIGGER_GROUP`) |eral_ci NULL DEFAULT NULL,\n PRIMARY KEY (`SCHED_NAME`, `TRIGGER_NAME`, `TRIGGER_GROUP`) USING BTREE|","USING|,|","CREATE TABLE `qrtz_cron_triggers` (\n `SCHED_NAME` varchar(120) CHARACTER SET utf8 COLLATE utf8_general_ci NOT NULL,\n `TRIGGER_NAME` varchar(190) CHARACTER SET utf8 COLLATE utf8_general_ci NOT NULL,\n `TRIGGER_GROUP` varchar(190) CHARACTER SET utf8 COLLATE utf8_general_ci NOT NULL,\n `CRON_EXPRESSION` varchar(120) CHARACTER SET utf8 COLLATE utf8_general_ci NOT NULL,\n `TIME_ZONE_ID` varchar(80) CHARACTER SET utf8 COLLATE utf8_general_ci NULL DEFAULT NULL,\n PRIMARY KEY (`SCHED_NAME`, `TRIGGER_NAME`, `TRIGGER_GROUP`) USING BTREE,\n CONSTRAINT `qrtz_cron_triggers_ibfk_1` FOREIGN KEY (`SCHED_NAME`, `TRIGGER_NAME`, `TRIGGER_GROUP`) REFERENCES `qrtz_triggers` (`SCHED_NAME`, `TRIGGER_NAME`, `TRIGGER_GROUP`) ON DELETE RESTRICT ON UPDATE RESTRICT\n) ENGINE = InnoDB CHARACTER SET = utf8 COLLATE = utf8_general_ci ROW_FORMAT = Dynamic;"
6480,422538,11,Expecting )|,"0) NOT NULL,\n `Price` int(11) NOT NULL,\n `SellCount` int(11) NOT NULL,\n PRIMARY KEY (`SellFood`) |\n `Price` int(11) NOT NULL,\n `SellCount` int(11) NOT NULL,\n PRIMARY KEY (`SellFood`) USING BTREE\n|",USING|)|,"-- 테이블 gimal2020.food 구조 내보내기\nCREATE TABLE IF NOT EXISTS `food` (\n `SellFood` varchar(50) NOT NULL,\n `Price` int(11) NOT NULL,\n `SellCount` int(11) NOT NULL,\n PRIMARY KEY (`SellFood`) USING BTREE\n) ENGINE=InnoDB DEFAULT CHARSET=utf8;"
3339,421217,566,Expecting )|,"T NULL COMMENT '所属域名',\n `target` int(1) DEFAULT '1' COMMENT '是否新页面打开1为是，0为否',\n PRIMARY KEY (`id`) |NT '所属域名',\n `target` int(1) DEFAULT '1' COMMENT '是否新页面打开1为是，0为否',\n PRIMARY KEY (`id`) USING BTREE\n|",USING|)|,"CREATE TABLE `ms_advertisement` (\n `id` int(11) NOT NULL AUTO_INCREMENT,\n `type` int(1) DEFAULT NULL COMMENT '类型，1广告代码，2用户自定义图片',\n `content` text COMMENT '广告代码',\n `titles` varchar(255) DEFAULT NULL COMMENT '标题',\n `info` varchar(255) DEFAULT NULL COMMENT '详细信息',\n `url` varchar(255) DEFAULT NULL COMMENT '外链',\n `position_id` int(11) DEFAULT NULL COMMENT '广告位',\n `add_time` int(10) DEFAULT NULL COMMENT '添加时间',\n `begin_time` int(10) DEFAULT NULL COMMENT '开始时间',\n `end_time` int(10) DEFAULT NULL COMMENT '结束时间',\n `status` int(1) DEFAULT '1' COMMENT '状态1开启，0关闭',\n `host` varchar(255) DEFAULT NULL COMMENT '所属域名',\n `target` int(1) DEFAULT '1' COMMENT '是否新页面打开1为是，0为否',\n PRIMARY KEY (`id`) USING BTREE\n) ENGINE=MyISAM AUTO_INCREMENT=21 DEFAULT CHARSET=utf8 ROW_FORMAT=DYNAMIC COMMENT='广告表';"
840,420204,21,Expecting )|,"OMMENT '认证结果编码',\n `create_time` datetime(0) NULL DEFAULT NULL COMMENT '创建时间',\n PRIMARY KEY (`id`) |编码',\n `create_time` datetime(0) NULL DEFAULT NULL COMMENT '创建时间',\n PRIMARY KEY (`id`) USING BTREE\n|",USING|)|,"CREATE TABLE `t_b_binding_record` (\n `id` varchar(36) CHARACTER SET utf8 COLLATE utf8_bin NOT NULL DEFAULT '' COMMENT '主键',\n `personal_transaction_code` varchar(50) CHARACTER SET utf8 COLLATE utf8_bin NULL DEFAULT NULL COMMENT '个人交易码',\n `unified_transaction_code` varchar(50) CHARACTER SET utf8 COLLATE utf8_bin NULL DEFAULT NULL COMMENT '机构交易码',\n `user_id` varchar(36) CHARACTER SET utf8 COLLATE utf8_bin NULL DEFAULT NULL COMMENT '用户id',\n `org_id` varchar(36) CHARACTER SET utf8 COLLATE utf8_bin NULL DEFAULT NULL COMMENT '机构id',\n `platform_code` varchar(11) CHARACTER SET utf8 COLLATE utf8_bin NULL DEFAULT NULL COMMENT '平台编码',\n `binding_type` varchar(2) CHARACTER SET 

### enum

In [161]:
no_using = no_asc[
    ~((no_asc.good2_file_parse_error_highlight.str.contains('USING',case=False,regex=False)) &
    ((no_asc.original.str.contains('BTREE',case=False,regex=False)) |
    ((no_asc.original.str.contains('INDEX',case=False,regex=False)))))
]
enum_check = no_using[
    (no_using.good2_file_parse_error_highlight == 'enum|')
]
print(get_stmt_and_file_count(enum_check))
#dot_check.sample(frac=0.6).head()

[########################################] | 100% Completed | 10.20 s
[########################################] | 100% Completed | 12.14 s
(42452, 12295)


In [160]:
enum_check.sample(frac=0.6).head()

[########################################] | 100% Completed | 395.31 ms


,file_id,statement_id,good2_parse_error,good2_file_parse_error_start_context,good2_file_parse_error_highlight,original
2629,420906,994,Expecting )|Invalid expression / Unexpected token|,"ULL,\n `auditor_id` varchar(32) default NULL,\n `audit_date` datetime default NULL,\n `audit_state` |",enum|,"CREATE TABLE `t_split` (\n `split_id` varchar(32) NOT NULL,\n `split_code` varchar(32) default NULL,\n `warehouse_id` varchar(32) default NULL,\n `split_date` datetime default NULL,\n `batch` int(11) default NULL,\n `sum_indirect_costs` decimal(16,4) default NULL,\n `department` varchar(32) default NULL,\n `auditor_id` varchar(32) default NULL,\n `audit_date` datetime default NULL,\n `audit_state` enum('authstr','failure','pass') default NULL,\n `audit_remark` varchar(255) default NULL,\n `enabled` enum('enable','disable') default NULL,\n `remark` varchar(255) default NULL,\n `creater` varchar(32) default NULL,\n `create_time` datetime default NULL,\n `updater` varchar(32) default NULL,\n `update_time` datetime default NULL,\n PRIMARY KEY (`split_id`)\n) ENGINE=InnoDB DEFAULT CHARSET=utf8;"
8212,422915,27,Expecting )|Invalid expression / Unexpected token|,"n` varchar(512) NOT NULL,\n `menu_image_name` varchar(50) CHARACTER SET latin1 NOT NULL,\n `status` |",enum|,"-- --------------------------------------------------------\n\n--\n-- Table structure for table `dn_menu`\n--\n\nCREATE TABLE IF NOT EXISTS `dn_menu` (\n `menu_id` bigint(20) NOT NULL AUTO_INCREMENT,\n `menu_name` varchar(256) NOT NULL,\n `punch_line` varchar(256) NOT NULL,\n `description` varchar(512) NOT NULL,\n `menu_image_name` varchar(50) CHARACTER SET latin1 NOT NULL,\n `status` enum('Active','Inactive') CHARACTER SET latin1 NOT NULL DEFAULT 'Active',\n PRIMARY KEY (`menu_id`)\n) ENGINE=InnoDB DEFAULT CHARSET=utf8 AUTO_INCREMENT=50 ;"
2615,420906,966,Expecting )|Invalid expression / Unexpected token|,"voice_number` varchar(20) default NULL,\n `invoice_date` datetime default NULL,\n `invoice_type_id` |",enum|,"CREATE TABLE `t_sell_invoice` (\n `invoice_id` varchar(32) NOT NULL,\n `invoice_code` varchar(50) default NULL,\n `receivale_id` varchar(32) default NULL,\n `client_id` varchar(32) default NULL,\n `client_contact_id` varchar(32) default NULL,\n `phone` varchar(15) default NULL,\n `client_address` varchar(255) default NULL,\n `invoice_account` varchar(32) default NULL,\n `invoice_number` varchar(20) default NULL,\n `invoice_date` datetime default NULL,\n `invoice_type_id` enum('specialvat','generalvat','generaltax','ordinarytax','receipt') default NULL,\n `Invoice_start` varchar(50) default NULL,\n `tariff` varchar(50) default NULL,\n `unbilled_amount` decimal(16,4) default NULL,\n `invoice_amount` decimal(16,4) default NULL,\n `deduction_date` decimal(16,4) default NULL,\n `ticket_date` decimal(16,4) default NULL,\n `invoice_collector` varchar(32) default NULL,\n `backwash_state` enum('backwash0','backwash1') default NULL,\n `enabled` enum('enable','disable') default NULL,\n `summary` varchar(255) default NULL,\n `remark` varchar(255) default NULL,\n `audit_state` enum('authstr','pass','failure') default NULL,\n `auditor` varchar(32) default NULL,\n `audit_date` datetime default NULL,\n `audit_remark` varchar(255) default NULL,\n `creater` varchar(32) default NULL,\n `create_time` datetime default NULL,\n `updater` varchar(32) default NULL,\n `update_time` datetime default NULL,\n PRIMARY KEY (`invoice_id`)\n) ENGINE=InnoDB DEFAULT CHARSET=utf8;"
10100,423785,34,Expecting )|Invalid expression / Unexpected token|,"archar(200) COLLATE utf8_unicode_ci NOT NULL,\n `page_priority` int(11) DEFAULT NULL,\n `page_type` |",enum|,"-- --------------------------------------------------------\n\n--\n-- Table structure for table `pages`\n--\n\nCREATE TABLE IF NOT EXISTS `pages` (\n `page_id` int(3) NOT NULL AUTO_INCREMENT,\n `page_url` varchar(400) COLLATE utf8_unicode_ci NOT NULL,\n `page_title` varchar(250) COLLATE utf8_unicode_ci NOT NULL,\n `page_short_description` varchar(5

### Unsigned again

In [162]:
no_enum = no_using[
    ~(no_using.good2_file_parse_error_highlight == 'enum|')
]

unsigned_again_check = no_enum[
    (no_enum.good2_file_parse_error_highlight.str.contains('UNSIGNED',case=False,regex=False))
]

print(get_stmt_and_file_count(unsigned_again_check))
#dot_check.sample(frac=0.6).head()

[########################################] | 100% Completed | 12.37 s
[########################################] | 100% Completed | 13.50 s
(29945, 2100)


In [163]:
unsigned_again_check.sample(frac=0.6).head()

[########################################] | 100% Completed | 258.04 ms


,file_id,statement_id,good2_parse_error,good2_file_parse_error_start_context,good2_file_parse_error_highlight,original
5996,422343,15,Expecting )|Invalid expression / Unexpected token|,"not null,\n cmci_sPrescripName varchar(96) not null,\n\tcmci_uSkillLevel tinyint\t|",unsigned|,"#查询生活技能代工订单用到配置表中信息，生活技能等级、产出物品等级\ncreate table tbl_contract_manufacture_cfg_info(\n\tcmci_sSkillName varchar(96) not null,\n cmci_sDirection varchar(96) not null,\n cmci_sPrescripName varchar(96) not null,\n\tcmci_uSkillLevel tinyint\tunsigned not null,\n\tcmci_uProductLevel tinyint\tunsigned not null,\n\n\tkey(cmci_sSkillName),\n\tkey(cmci_sDirection),\n\tkey(cmci_sPrescripName)\n)engine = memory;"
12446,424605,38,Expecting )|Invalid expression / Unexpected token|,"EATE TABLE `#__session` (\n `session_id` varchar(200) NOT NULL default '',\n `client_id` tinyint(3) |",unsigned|,"# -------------------------------------------------------\n\n#\n# Table structure for table `#__session`\n#\n\nCREATE TABLE `#__session` (\n `session_id` varchar(200) NOT NULL default '',\n `client_id` tinyint(3) unsigned NOT NULL default '0',\n `guest` tinyint(4) unsigned default '1',\n `time` varchar(14) default '',\n `data` mediumtext default NULL,\n `userid` int(11) default '0',\n `username` varchar(150) default '',\n `usertype` varchar(50) default '',\n PRIMARY KEY (`session_id`),\n KEY `whosonline` (`guest`,`usertype`),\n KEY `userid` (`userid`),\n KEY `time` (`time`)\n) DEFAULT CHARSET=utf8;"
5990,422343,7,Expecting )|Invalid expression / Unexpected token|,#售出到商店的堆叠物品id\ncreate table tbl_player_sell_item_id(\n psii_nId bigint |,unsigned|,"#售出到商店的堆叠物品id\ncreate table tbl_player_sell_item_id(\n psii_nId bigint unsigned not null,\n psii_nItemId bigint unsigned not null,\n unique key(psii_nItemId),\n key(psii_nId),\n foreign key (psii_nId)\treferences tbl_npcshop_player_sell_item(npsi_nId) on update cascade on delete cascade,\n foreign\tkey(psii_nItemId)\treferences tbl_item_static(is_uId) on\tupdate cascade on delete cascade\n)engine = innodb;"
5893,422238,10,Expecting )|Invalid expression / Unexpected token|,#DEPRECATED??\nCREATE TABLE output (\n job_id int(10) |,unsigned|,"#DEPRECATED??\nCREATE TABLE output (\n job_id int(10) unsigned DEFAULT '0' NOT NULL,\n output_name varchar(40) DEFAULT '' NOT NULL,\n PRIMARY KEY (job_id, output_name)\n);"
5993,422343,12,Expecting )|Invalid expression / Unexpected token|,##### order\ncreate table tbl_contract_manufacture_order_static(\t\t\t\n\tcmo_uId\t\t\t bigint |,unsigned|,"##### order\ncreate table tbl_contract_manufacture_order_static(\t\t\t\n\tcmo_uId\t\t\t bigint unsigned\t\tnot null \tauto_increment,\t\t#order identifier\n\tcmo_tCreateTime\t\tdatetime \t\t not null,\t\t\t\t#create time\n\tprimary key(cmo_uId)\n)engine=innodb;"


### CREATE -> db, go, delimiter

In [166]:
no_unsigned_again = no_enum[
    (~no_enum.good2_file_parse_error_highlight.str.contains('UNSIGNED',case=False,regex=False))
]

create_check = no_unsigned_again[
    (no_unsigned_again.good2_file_parse_error_highlight.str.contains('CREATE|',case=False,regex=False))
]
print(get_stmt_and_file_count(create_check))
#dot_check.sample(frac=0.6).head()

[########################################] | 100% Completed | 12.08 s
[########################################] | 100% Completed | 12.55 s
(59234, 18302)


In [174]:
create_db_check = no_unsigned_again[
    (no_unsigned_again.good2_file_parse_error_highlight.str.contains('CREATE|',case=False,regex=False)) &
    (no_unsigned_again.original.str.contains('CREATE DATABASE',case=False,regex=False))
]
print(get_stmt_and_file_count(create_db_check))

[########################################] | 100% Completed | 12.85 s
[########################################] | 100% Completed | 11.46 s
(1659, 1605)


In [168]:
create_go_check = no_unsigned_again[
    (no_unsigned_again.good2_file_parse_error_highlight.str.contains('CREATE|',case=False,regex=False)) &
    (no_unsigned_again.good2_file_parse_error_start_context.str.contains('GO',case=False,regex=False))
]
print(get_stmt_and_file_count(create_go_check))

[########################################] | 100% Completed | 11.96 s
[########################################] | 100% Completed | 12.39 s
(20201, 3180)


In [169]:
create_delimiter_check = no_unsigned_again[
    (no_unsigned_again.good2_file_parse_error_highlight.str.contains('CREATE|',case=False,regex=False)) &
    (no_unsigned_again.good2_file_parse_error_start_context.str.contains('DELIMITER',case=False,regex=False))
]
print(get_stmt_and_file_count(create_delimiter_check))

[########################################] | 100% Completed | 11.40 s
[########################################] | 100% Completed | 11.35 s
(12088, 3207)


In [170]:
create_delimiter_check2 = no_unsigned_again[
    (no_unsigned_again.good2_file_parse_error_highlight.str.contains('CREATE|',case=False,regex=False)) &
    (no_unsigned_again.good2_file_parse_error_start_context.str.contains('DELIMITER',case=False,regex=False)) &
    (
        (no_unsigned_again.original.str.contains('CREATE TRIGGER',case=False,regex=False)) |
        (no_unsigned_again.original.str.contains('CREATE PROECDURE',case=False,regex=False))
    )
]
print(get_stmt_and_file_count(create_delimiter_check2))

[########################################] | 100% Completed | 13.32 s
[########################################] | 100% Completed | 13.16 s
(4561, 1699)


In [177]:
create_go_check.sample(frac=0.6).head(10)

[########################################] | 100% Completed | 419.04 ms


,file_id,statement_id,good2_parse_error,good2_file_parse_error_start_context,good2_file_parse_error_highlight,original
4680,421735,69,Invalid expression / Unexpected token|,GO\n|,CREATE|,"GO\nCREATE TABLE active_passive_rec_m(\n\t[id] INTEGER Primary Key,\n\t[mid] INTEGER NOT NULL,\n\t[rid] INTEGER NOT NULL,\n\t[a] float,\n\t[b] float,\n\t[c] float,\n\t[d] float,\n\t[var_a] float,\n\t[var_b] float,\n\t[var_c] float,\n\t[var_d] float,\n\t[covar_ab] float,\n\t[covar_ac] float,\n\t[covar_ad] float,\n\t[covar_bc] float,\n\t[covar_bd] float,\n\t[covar_cd] float,\n\t[sigma_x] float,\n\t[cal_curve_equation] int,\n\t[lower_mass_limit] float,\n\t[upper_mass_limit] float,\n\tFOREIGN KEY(mid) REFERENCES measurements(id) on DELETE CASCADE,\n\tFOREIGN KEY(rid) REFERENCES results_active_passive_rec(id) on DELETE CASCADE\n);"
3454,421242,162,Invalid expression / Unexpected token|,GO\n--1 Create a scalar function\n|,CREATE|,"GO\n--1 Create a scalar function\nCREATE OR ALTER FUNCTION dbo.udf_GetSalesCount(@ProductID INT, @Year INT = 0)\nRETURNS INT AS \nBEGIN\n\tDECLARE @Count INT;\n\tIF @Year = 0 BEGIN \n\t\tSELECT @Count = COALESCE(SUM(OrderQty),0)\n\t\tFROM Sales.SalesOrderDetail \n\t\tWHERE ProductID = @ProductID;\n\tEND ELSE \n\t\tSELECT @Count = COALESCE(SUM(OrderQty),0)\n\t\tFROM Sales.SalesOrderDetail AS SOD \n\t\tJOIN Sales.SalesOrderHeader AS SOH \n\t\tON SOD.SalesOrderID = SOH.SalesOrderID\n\t\tWHERE ProductID = @ProductID\n\t\t\tAND YEAR(OrderDate) = @Year;\n\tRETURN @Count;\nEND\n\nGO\n--2 Switch to 2017 compat mode\nALTER DATABASE [AdventureWorks2019] SET COMPATIBILITY_LEVEL = 140\nGO\n\n--3 Use the function\nSELECT ProductID, Name, ListPrice, dbo.udf_GetSalesCount(ProductID,0) AS QtySold\nFROM Production.Product\nWHERE ProductSubcategoryID = 1;\n\n--4 Switch to 2019 Compat\nALTER DATABASE [AdventureWorks2019] SET COMPATIBILITY_LEVEL = 150\nGO\n\n--5 Use the function\nSELECT ProductID, Name, ListPrice, dbo.udf_GetSalesCount(ProductID,0) AS QtySold\nFROM Production.Product\nWHERE ProductSubcategoryID = 1;\n\n \n--Listing 13-23. Inline multi-statement table-valued function \n--Create a function\nGO\nCREATE OR ALTER FUNCTION dbo.getColors()\nRETURNS @Colors TABLE \n(\n\tColor varchar(20)\n)\nAS\nBEGIN\n\tINSERT INTO @Colors(Color) \n\tSELECT DISTINCT Color \n\tFROM Production.Product \n\n\tINSERT INTO @Colors(Color)\n\tVALUES('Lime'),('Turquoise');\n\t\n\tRETURN;\nEND\nGO"
4649,421735,38,Invalid expression / Unexpected token|,GO\n|,CREATE|,"GO\nCREATE TABLE items(\n\t[item_id] INTEGER Primary Key,\n\t[item_name] nvarchar(256) NULL,\n\t[material_type_id] nvarchar(50) NULL,\n\t[isotopics_id] nvarchar(256) NULL,\n\t[stratum_id] nvarchar(256) NULL,\n\t[inventory_change_code] nvarchar(10) NULL,\n\t[io_code] nvarchar(10) NULL,\n\t[declared_mass_entry] float NULL,\n\t[declared_u_mass_entry] float NULL,\n\t[length_entry] float NULL,\n\t[mba] nvarchar(256) NULL\n);"
7728,422768,292,Invalid expression / Unexpected token|,GO\n\n|,CREATE|,"GO\n\nCREATE TABLE TimesheetPayment\n(\n\tTimesheetPaymentID\t\t INT NOT NULL,\n\tTimesheetCompanyID\t\t INT NOT NULL,\n\tTimesheetPaymentDate\t\t DATETIME NOT NULL,\n\tTimesheetPayment\t\t NUMERIC(9,2) NOT NULL,\n\tActive\t\t BIT DEFAULT ((1)) NOT NULL,\n\tcreate_dt\t\t DATETIME NOT NULL\n\n);"
7474,422768,38,Invalid expression / Unexpected token|,GO\n\n|,CREATE|,"GO\n\nCREATE TABLE CreditCard\n(\n\tCreditCardID\t\tIDENTITY (1001,1) INT NOT NULL,\n\tUserID\t\t INT NOT NULL,\n\tCreditCardName\t\t VARCHAR(50) NOT NULL,\n\tCreditLimit\t\t NUMERIC(9,2) NOT NULL,\n\tMinimumPayment\t\t NUMERIC(9,2) NULL,\n\tCreditAvailable\t\t NUMERIC(9,2) NULL,\n\tClosingDate\t\t TINYINT NULL,\n\tDueDate\t\t TINYINT NULL,\n\tPercentage\t\t NUMERIC(9,2) NULL,\n\tPayments\t\t NUMERIC(9,2) NULL,\n\tcreate_dt\t\t DATETIME DEFAULT (getdate()) NOT NULL\n\n);"
7722,422768,286,Invalid expression / Unexpected token|,GO\n\n|,CREATE|,"GO\n\nCREATE TABLE TimesheetCompany\n(\n\tTimeSheetCompanyID\t\tIDENTITY (1001,1) INT NOT NULL,\n\tUserID\t

### DROP

In [180]:
no_create_db_go_del = no_unsigned_again[
    ~((no_unsigned_again.good2_file_parse_error_highlight.str.contains('CREATE|',case=False,regex=False)) &
    (
        (no_unsigned_again.good2_file_parse_error_start_context.str.contains('DELIMITER',case=False,regex=False)) |
        (no_unsigned_again.good2_file_parse_error_start_context.str.contains('GO',case=False,regex=False)) |
        (no_unsigned_again.original.str.contains('DATABASE',case=False,regex=False))
    ))
]

drop_ctr_check = no_create_db_go_del[
    (no_create_db_go_del.good2_file_parse_error_highlight.str.contains('drop|',case=False,regex=False)) &
    (no_create_db_go_del.original.str.contains('DROP CONSTRAINT',case=False,regex=False))
]

print(get_stmt_and_file_count(drop_ctr_check))
#dot_check.sample(frac=0.6).head()


[########################################] | 100% Completed | 12.44 s
[########################################] | 100% Completed | 15.60 s
(32030, 3276)


In [181]:
drop_ctr_check.sample(frac=0.6).head()

[########################################] | 100% Completed | 528.46 ms


,file_id,statement_id,good2_parse_error,good2_file_parse_error_start_context,good2_file_parse_error_highlight,original
1730,420551,26,Invalid expression / Unexpected token|,alter table TRSREPRINT |,drop|,alter table TRSREPRINT drop constraint FK_RECREP_RECEIPTID;
3829,421352,58,Invalid expression / Unexpected token|,ALTER TABLE PRODUCTS |,DROP|,ALTER TABLE PRODUCTS DROP CONSTRAINT PRODUCTS_FK_2;
8294,422938,32,Invalid expression / Unexpected token|,ALTER TABLE test_run_y2007_m05_wk2 |,DROP|,ALTER TABLE test_run_y2007_m05_wk2 DROP CONSTRAINT test_run_y2007_m05_wk2_test_run_command_id_fkey;
8282,422938,20,Invalid expression / Unexpected token|,ALTER TABLE test_run_y2007_m02_wk5 |,DROP|,ALTER TABLE test_run_y2007_m02_wk5 DROP CONSTRAINT test_run_y2007_m02_wk5_test_run_command_id_fkey;
8267,422938,5,Invalid expression / Unexpected token|,ALTER TABLE test_run_y2006_m11_wk5 |,DROP|,ALTER TABLE test_run_y2006_m11_wk5 DROP CONSTRAINT test_run_y2006_m11_wk5_test_run_command_id_fkey;


### Mediumint

In [185]:
no_drop_ctr = no_create_db_go_del[
    ~((no_create_db_go_del.good2_file_parse_error_highlight.str.contains('drop|',case=False,regex=False)) &
    (no_create_db_go_del.original.str.contains('DROP CONSTRAINT',case=False,regex=False)))
]

mediumint_check = no_drop_ctr[
    (no_drop_ctr.good2_file_parse_error_highlight.str.contains('mediumint|',case=False,regex=False))
]

print(get_stmt_and_file_count(mediumint_check))
#dot_check.sample(frac=0.6).head()

[########################################] | 100% Completed | 13.08 s
[########################################] | 100% Completed | 15.75 s
(40051, 5694)


In [184]:
mediumint_check.sample(frac=0.6).head()

[########################################] | 100% Completed | 447.15 ms


,file_id,statement_id,good2_parse_error,good2_file_parse_error_start_context,good2_file_parse_error_highlight,original
15694,425909,13,Expecting )|Invalid expression / Unexpected token|,CREATE TABLE `aa_actions` (\n `aaid` |,mediumint|,"CREATE TABLE `aa_actions` (\n `aaid` mediumint(8) unsigned NOT NULL DEFAULT '0',\n `rank` tinyint(3) unsigned NOT NULL DEFAULT '0',\n `reuse_time` mediumint(8) unsigned NOT NULL DEFAULT '0',\n `spell_id` mediumint(8) unsigned NOT NULL DEFAULT '0',\n `cast_time` mediumint(8) unsigned NOT NULL DEFAULT '0',\n `target` tinyint(3) unsigned NOT NULL DEFAULT '0',\n `nonspell_action` tinyint(3) unsigned NOT NULL DEFAULT '0',\n `nonspell_mana` mediumint(8) unsigned NOT NULL DEFAULT '0',\n `nonspell_duration` mediumint(8) unsigned NOT NULL DEFAULT '0',\n `redux_aa` mediumint(8) unsigned NOT NULL DEFAULT '0',\n `redux_rate` tinyint(4) NOT NULL DEFAULT '0',\n PRIMARY KEY (`aaid`,`rank`)\n) ENGINE=MyISAM DEFAULT CHARSET=latin1;"
9537,423504,618,Expecting )|Invalid expression / Unexpected token|,CREATE TABLE `reference_loot_template` (\n `Entry` |,mediumint|,"CREATE TABLE `reference_loot_template` (\n `Entry` mediumint(8) unsigned NOT NULL DEFAULT '0',\n `Item` mediumint(8) unsigned NOT NULL DEFAULT '0',\n `Reference` mediumint(8) unsigned NOT NULL DEFAULT '0',\n `Chance` float NOT NULL DEFAULT '100',\n `QuestRequired` tinyint(1) NOT NULL DEFAULT '0',\n `LootMode` smallint(5) unsigned NOT NULL DEFAULT '1',\n `GroupId` tinyint(3) unsigned NOT NULL DEFAULT '0',\n `MinCount` tinyint(3) unsigned NOT NULL DEFAULT '1',\n `MaxCount` tinyint(3) unsigned NOT NULL DEFAULT '1',\n `Comment` varchar(255) DEFAULT NULL,\n PRIMARY KEY (`Entry`,`Item`)\n) ENGINE=MyISAM DEFAULT CHARSET=utf8 ROW_FORMAT=FIXED COMMENT='Loot System';"
8517,423036,13,Expecting )|Invalid expression / Unexpected token|,CREATE TABLE `sp_address` (\n `id` |,mediumint|,"CREATE TABLE `sp_address` (\n `id` mediumint(8) unsigned NOT NULL AUTO_INCREMENT COMMENT '地址ID',\n `name` varchar(10) NOT NULL DEFAULT '' COMMENT '收货人',\n `phone` char(11) NOT NULL DEFAULT '' COMMENT '联系电话',\n `prov` varchar(20) NOT NULL DEFAULT '' COMMENT '省',\n `city` varchar(20) NOT NULL DEFAULT '' COMMENT '市',\n `dist` varchar(20) NOT NULL DEFAULT '' COMMENT '区',\n `address` varchar(150) NOT NULL DEFAULT '' COMMENT '详细地址',\n `email` varchar(50) NOT NULL DEFAULT '' COMMENT '邮箱地址',\n `member_id` mediumint(8) unsigned NOT NULL COMMENT '会员ID',\n PRIMARY KEY (`id`)\n) ENGINE=InnoDB AUTO_INCREMENT=4 DEFAULT CHARSET=utf8 COMMENT='地址表';"
10840,424157,22,Expecting )|Invalid expression / Unexpected token|,"`Title`\t\t\tvarchar(50) not null,\n `CategoryID`\ttinyint,\n `Duration`\t\tfloat,\n `CreatorID`\t\t|",mediumint|,"create table if not exists `Exam`\n(\n\t`ExamID`\t\ttinyint primary key auto_increment not null,\n `Code`\t\t\tvarchar(50) not null,\n `Title`\t\t\tvarchar(50) not null,\n `CategoryID`\ttinyint,\n `Duration`\t\tfloat,\n `CreatorID`\t\tmediumint,\n `CreateDate`\tdatetime,\n \n CONSTRAINT fk_ex_acc FOREIGN KEY (`CreatorID`)\n\tREFERENCES `account`(`AccountID`),\n\tCONSTRAINT fk_ex_cate FOREIGN KEY (`CategoryID`) \n\tREFERENCES `CategoryQuestion` (`CategoryID`)\n \n \n);"
9526,423504,453,Expecting )|Invalid expression / Unexpected token|,CREATE TABLE `npc_trainer` (\n `entry` |,mediumint|,"CREATE TABLE `npc_trainer` (\n `entry` mediumint(8) unsigned NOT NULL DEFAULT '0',\n `spell` mediumint(8) NOT NULL DEFAULT '0',\n `spellcost` int(10) unsigned NOT NULL DEFAULT '0',\n `reqskill` smallint(5) unsigned NOT NULL DEFAULT '0',\n `reqskillvalue` smallint(5) unsigned NOT NULL DEFAULT '0',\n `reqlevel` tinyint(3) unsigned NOT NULL DEFAULT '0',\n PRIMARY KEY (`entry`,`spell`)\n) ENGINE=MyISAM DEFAULT CHARSET=utf8;"


### variations of text

In [186]:
no_mediumint = no_drop_ctr[
    (~no_drop_ctr.good2_file_parse_error_highlight.str.contains('mediumint|',case=False,regex=False))
]

var_text_check = no_mediumint[
    (no_mediumint.good2_file_parse_error_highlight.str.contains('longtext',case=False,regex=False)) |
    (no_mediumint.good2_file_parse_error_highlight.str.contains('mediumtext',case=False,regex=False)) |
    (no_mediumint.good2_file_parse_error_highlight.str.contains('tinytext',case=False,regex=False))
]
print(get_stmt_and_file_count(var_text_check))
#dot_check.sample(frac=0.6).head()

[########################################] | 100% Completed | 19.48 s
[########################################] | 100% Completed | 18.60 s
(68750, 20394)


In [187]:
var_text_check.sample(frac=0.6).head()

[########################################] | 100% Completed | 415.35 ms


,file_id,statement_id,good2_parse_error,good2_file_parse_error_start_context,good2_file_parse_error_highlight,original
4258,421481,18,Expecting )|Invalid expression / Unexpected token|,"\n `type` varchar(20) NOT NULL DEFAULT '',\n `type_txt` varchar(128) NOT NULL DEFAULT '',\n `setup` |",mediumtext|,"CREATE TABLE `db_field` (\n `id` int(11) NOT NULL AUTO_INCREMENT,\n `moduleid` tinyint(3) NOT NULL DEFAULT '0',\n `field` varchar(20) NOT NULL DEFAULT '',\n `name` varchar(50) NOT NULL DEFAULT '',\n `tips` varchar(150) NOT NULL DEFAULT '',\n `type` varchar(20) NOT NULL DEFAULT '',\n `type_txt` varchar(128) NOT NULL DEFAULT '',\n `setup` mediumtext,\n `required` tinyint(1) NOT NULL DEFAULT '0',\n `issystem` tinyint(1) NOT NULL DEFAULT '0',\n `listorder` int(10) NOT NULL DEFAULT '0',\n `status` tinyint(1) NOT NULL DEFAULT '0',\n `minlength` int(10) NOT NULL DEFAULT '0',\n `maxlength` int(10) NOT NULL DEFAULT '0',\n `pattern` varchar(128) NOT NULL DEFAULT '',\n `ispost` tinyint(1) NOT NULL DEFAULT '1',\n `validate` varchar(255) NOT NULL DEFAULT '',\n PRIMARY KEY (`id`),\n KEY `moduleid` (`moduleid`)\n) ENGINE=MyISAM DEFAULT CHARSET=utf8;"
4316,421525,87,Expecting )|Invalid expression / Unexpected token|,"auto_increment,\n `hl7_source` int(11) NOT NULL default '0',\n `hl7_source_key` text,\n `hl7_data` |",mediumtext|,"CREATE TABLE `hl7_in_error` (\n `hl7_in_error_id` int(11) NOT NULL auto_increment,\n `hl7_source` int(11) NOT NULL default '0',\n `hl7_source_key` text,\n `hl7_data` mediumtext NOT NULL,\n `error` varchar(255) NOT NULL default '',\n `error_details` text,\n `date_created` datetime NOT NULL default '0000-00-00 00:00:00',\n PRIMARY KEY (`hl7_in_error_id`)\n) ENGINE=InnoDB DEFAULT CHARSET=utf8;"
2015,420645,66,Expecting )|Invalid expression / Unexpected token|,".course_id\n `ci_id` int(11) NOT NULL, -- foreign key to link to courses_items.ci_id\n `staff_note` |",mediumtext|,"CREATE TABLE `course_reserves` (\n `cr_id` int(11) NOT NULL AUTO_INCREMENT,\n `course_id` int(11) NOT NULL, -- foreign key to link to courses.course_id\n `ci_id` int(11) NOT NULL, -- foreign key to link to courses_items.ci_id\n `staff_note` mediumtext, -- staff only note\n `public_note` mediumtext, -- public, OPAC visible note\n `timestamp` timestamp NOT NULL DEFAULT CURRENT_TIMESTAMP ON UPDATE CURRENT_TIMESTAMP,\n PRIMARY KEY (`cr_id`),\n UNIQUE KEY `pseudo_key` (`course_id`,`ci_id`),\n KEY `course_id` (`course_id`)\n) ENGINE=InnoDB DEFAULT CHARSET=utf8 COLLATE=utf8_unicode_ci;"
12964,424902,118,Expecting )|Invalid expression / Unexpected token|,"CREATE TABLE `django_session` (\n `session_key` varchar(40) NOT NULL,\n `session_data` |",longtext|,"CREATE TABLE `django_session` (\n `session_key` varchar(40) NOT NULL,\n `session_data` longtext NOT NULL,\n `expire_date` datetime(6) NOT NULL,\n PRIMARY KEY (`session_key`),\n KEY `django_session_de54fa62` (`expire_date`)\n) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4;"
8585,423052,231,Expecting )|Invalid expression / Unexpected token|,"200) NOT NULL,\n `Address` varchar(45) DEFAULT NULL,\n `UserName` varchar(45) NOT NULL,\n `HeadImg` |",longtext|,"CREATE TABLE `wechatuser` (\n `ID` int(11) NOT NULL AUTO_INCREMENT,\n `OpenID` varchar(200) NOT NULL,\n `Address` varchar(45) DEFAULT NULL,\n `UserName` varchar(45) NOT NULL,\n `HeadImg` longtext NOT NULL,\n `Sex` int(11) DEFAULT NULL,\n `GPS` varchar(45) DEFAULT NULL,\n `Quota` int(11) DEFAULT '0',\n `BeginTime` timestamp NULL DEFAULT NULL,\n `Credit` float DEFAULT '0',\n `IsFollowed` int(11) DEFAULT NULL,\n PRIMARY KEY (`ID`)\n) ENGINE=InnoDB AUTO_INCREMENT=11 DEFAULT CHARSET=utf8;"


### ADD

In [193]:
no_var_text = no_mediumint[
    ~((no_mediumint.good2_file_parse_error_highlight.str.contains('longtext',case=False,regex=False)) |
    (no_mediumint.good2_file_parse_error_highlight.str.contains('mediumtext',case=False,regex=False)) |
    (no_mediumint.good2_file_parse_error_highlight.str.contains('tinytext',case=False,regex=False)))
]

add_chk = no_var_text[
    (no_var_text.good2_file_parse_error_highlight.str.contains('add',case=False,regex=False))
]

print(get_stmt_and_file_count(add_chk))
#dot_check.sample(frac=0.6).head()

[########################################] | 100% Completed | 16.08 s
[########################################] | 100% Completed | 15.65 s
(77460, 11020)


In [192]:
add_idx_chk = no_var_text[
    (no_var_text.good2_file_parse_error_highlight.str.contains('add',case=False,regex=False)) &
    (no_var_text.original.str.contains('ALTER TABLE',case=False,regex=False)) &
    (no_var_text.original.str.contains('ADD INDEX',case=False,regex=False))
]
print(get_stmt_and_file_count(add_idx_chk))

[########################################] | 100% Completed | 22.95 s
[########################################] | 100% Completed | 21.24 s
(30374, 1691)


In [194]:
add_uq_chk = no_var_text[
    (no_var_text.good2_file_parse_error_highlight.str.contains('add',case=False,regex=False)) &
    (no_var_text.original.str.contains('ALTER TABLE',case=False,regex=False)) &
    (no_var_text.original.str.contains('ADD UNIQUE',case=False,regex=False))
]
print(get_stmt_and_file_count(add_uq_chk))

[########################################] | 100% Completed | 18.12 s
[########################################] | 100% Completed | 18.46 s
(3790, 1328)


In [196]:
add_ctr_chk = no_var_text[
    (no_var_text.good2_file_parse_error_highlight.str.contains('add',case=False,regex=False)) &
    (no_var_text.original.str.contains('ALTER TABLE',case=False,regex=False)) &
    (no_var_text.original.str.contains('ADD CONSTRAINT',case=False,regex=False))
]
print(get_stmt_and_file_count(add_ctr_chk))

[########################################] | 100% Completed | 13.69 s
[########################################] | 100% Completed | 14.54 s
(26046, 1307)


In [199]:
add_ctr_chk.sample(frac=0.6).head()

[########################################] | 100% Completed | 732.57 ms


,file_id,statement_id,good2_parse_error,good2_file_parse_error_start_context,good2_file_parse_error_highlight,original
433,419965,89,Invalid expression / Unexpected token|,alter table PASSPORTS \n |,add|,"alter table PASSPORTS \n add index FK6E63068170433D02 (TRAVELLER_ID), \n add constraint FK6E63068170433D02 \n foreign key (TRAVELLER_ID) \n references TRAVELLERS (ID);"
1540,420467,5,Invalid expression / Unexpected token|,--给员工表设置外键emp_dept_id(带级联置空)\nALTER TABLE emp\n|,ADD|,--给员工表设置外键emp_dept_id(带级联置空)\nALTER TABLE emp\nADD CONSTRAINTS emp_dept_id_fk FOREIGN KEY(emp_dept_id)\nREFERENCES dept (dept_id)\nON DELETE SET NULL;
436,419965,92,Invalid expression / Unexpected token|,alter table PEOPLE \n |,add|,"alter table PEOPLE \n add index FK8C75268F396AC1B2 (BIRTH_CITY_ID), \n add constraint FK8C75268F396AC1B2 \n foreign key (BIRTH_CITY_ID) \n references CITIES (ID);"
430,419965,86,Invalid expression / Unexpected token|,alter table MEDICALASSESSMENTS \n |,add|,"alter table MEDICALASSESSMENTS \n add index FK296ECEC070433D02 (TRAVELLER_ID), \n add constraint FK296ECEC070433D02 \n foreign key (TRAVELLER_ID) \n references TRAVELLERS (ID);"
2401,420818,63,Invalid expression / Unexpected token|,ol2SvcClass\n ADD CONSTRAINT FK_DiskPool2SvcClass_P FOREIGN KEY (Parent) REFERENCES DiskPool (id)\n |,ADD|,ALTER TABLE DiskPool2SvcClass\n ADD CONSTRAINT FK_DiskPool2SvcClass_P FOREIGN KEY (Parent) REFERENCES DiskPool (id)\n ADD CONSTRAINT FK_DiskPool2SvcClass_C FOREIGN KEY (Child) REFERENCES SvcClass (id);


### $FILE

In [202]:
no_add = no_var_text[
    ~((no_var_text.good2_file_parse_error_highlight.str.contains('add',case=False,regex=False)) &
    (no_var_text.original.str.contains('ALTER TABLE',case=False,regex=False)) &
    (
        (no_var_text.original.str.contains('ADD UNIQUE',case=False,regex=False)) |
        (no_var_text.original.str.contains('ADD CONSTRAINT',case=False,regex=False)) |
        (no_var_text.original.str.contains('ADD INDEX',case=False,regex=False))
    ))
]

file_thing_check = no_add[
    (no_add.original.str.contains('$FILE',case=False,regex=False))
]
print(get_stmt_and_file_count(file_thing_check))
#dot_check.sample(frac=0.6).head()    

[########################################] | 100% Completed | 23.99 s
[########################################] | 100% Completed | 28.12 s
(22263, 519)


### ADD AGAIN

In [209]:
no_file_thing = no_add[
    (~no_add.original.str.contains('$FILE',case=False,regex=False))
]

add_gain_check = no_file_thing[
    (no_file_thing.good2_file_parse_error_highlight.str.contains('add',case=False,regex=False)) &
    (no_file_thing.original.str.contains('ALTER TABLE',case=False,regex=False))
]
print(get_stmt_and_file_count(add_gain_check))
#dot_check.sample(frac=0.6).head()   

[########################################] | 100% Completed | 26.61 s
[########################################] | 100% Completed | 28.10 s
(29229, 5013)


In [207]:
add_gain_check.sample(frac=0.6).head() 

[########################################] | 100% Completed | 818.76 ms


,file_id,statement_id,good2_parse_error,good2_file_parse_error_start_context,good2_file_parse_error_highlight,original
4860,421744,84,Invalid expression / Unexpected token|,ALTER TABLE TWO_FACTOR_DEVICE_SERIAL |,ADD|,ALTER TABLE TWO_FACTOR_DEVICE_SERIAL ADD (\n CONSTRAINT TWO_FACTOR_DEVICE_SERIAL_R01\n FOREIGN KEY (USER_UUID)\n REFERENCES TWO_FACTOR_USER (UUID));
14284,425193,110,Invalid expression / Unexpected token|,"ALTER TABLE ""C##DBETAPA1"".""AUTH_USER"" |",ADD|,"ALTER TABLE ""C##DBETAPA1"".""AUTH_USER"" ADD CHECK (""IS_SUPERUSER"" IN (0,1)) ENABLE;"
4857,421744,70,Invalid expression / Unexpected token|,ALTER TABLE TWO_FACTOR_REPORT_ROLLUP |,ADD|,ALTER TABLE TWO_FACTOR_REPORT_ROLLUP ADD (\n CONSTRAINT TWO_FACTOR_REPORT_ROLLUP_R02\n FOREIGN KEY (CHILD_REPORT_UUID)\n REFERENCES TWO_FACTOR_REPORT (UUID));
937,420214,0,Invalid expression / Unexpected token|,-- PROFILE\n\nalter table DB_WCT.PROFILE |,add|,-- PROFILE\n\nalter table DB_WCT.PROFILE add (P_HARVESTER_TYPE varchar2(40));
2272,420733,10,Invalid expression / Unexpected token|,ALTER TABLE anuncios\n|,ADD|,ALTER TABLE anuncios\nADD fk_anuncio INT NOT NULL\nAFTER precio;


### {install:drop_tables} thing

In [215]:
no_add_again = no_file_thing[
    ~((no_file_thing.good2_file_parse_error_highlight.str.contains('add',case=False,regex=False)) &
    (no_file_thing.original.str.contains('ALTER TABLE',case=False,regex=False)))
]

install_thing_chk = no_add_again[
    (no_add_again.original.str.contains('{install:',case=False,regex=False))
]
print(get_stmt_and_file_count(install_thing_chk))
#dot_check.sample(frac=0.6).head()   

[########################################] | 100% Completed | 23.25 s
[########################################] | 100% Completed | 22.92 s
(9689, 204)


### INTO

In [218]:
no_install_thing = no_add_again[
    (~no_add_again.original.str.contains('{install:',case=False,regex=False))
]

into_chk = no_install_thing[
    (no_install_thing.good2_file_parse_error_highlight.str.contains('INTO',case=False,regex=False))
]
#print(get_stmt_and_file_count(install_thing_chk))
#dot_check.sample(frac=0.6).head()   

In [220]:
rem_into_chk = no_install_thing[
    (no_install_thing.good2_file_parse_error_highlight.str.contains('INTO',case=False,regex=False)) &
    (no_install_thing.original.str.contains('REM INSERTING',case=False,regex=False))
]
print(get_stmt_and_file_count(rem_into_chk))

[########################################] | 100% Completed | 31.23 s
[########################################] | 100% Completed | 26.26 s
(4286, 565)


In [221]:
insert_ignore_chk = no_install_thing[
    (no_install_thing.good2_file_parse_error_highlight.str.contains('INTO',case=False,regex=False)) &
    (no_install_thing.original.str.contains('INSERT IGNORE INTO',case=False,regex=False))
]
print(get_stmt_and_file_count(insert_ignore_chk))

[########################################] | 100% Completed | 29.32 s
[########################################] | 100% Completed | 32.79 s
(21471, 1626)


In [223]:
insert_ignore_chk.sample(frac=0.6).head()   

[########################################] | 100% Completed | 881.23 ms


,file_id,statement_id,good2_parse_error,good2_file_parse_error_start_context,good2_file_parse_error_highlight,original
89,419802,9,Invalid expression / Unexpected token|,INSERT IGNORE |,INTO|,"INSERT IGNORE INTO `cloud`.`configuration` VALUES ('Domain Defaults', 'DEFAULT', 'management-server', 'max.domain.volumes','-1','The default maximum number of volumes that can be created for a domain', '-1', NULL, NULL, 0);"
8061,422827,38,Invalid expression / Unexpected token|,INSERT IGNORE |,INTO|,"INSERT IGNORE INTO tiki_article_types(type,show_post_expire) VALUES ('Event','n');"
13046,424965,42,Invalid expression / Unexpected token|,--\n-- Dumping data for table `#__menu_types`\n--\n\nINSERT IGNORE |,INTO|,"--\n-- Dumping data for table `#__menu_types`\n--\n\nINSERT IGNORE INTO `#__menu_types` (`id`, `asset_id`, `menutype`, `title`, `description`, `client_id`) VALUES\n(1, 0, 'mainmenu', 'Main Menu', 'The main menu for the site', 0);"
85,419802,5,Invalid expression / Unexpected token|,INSERT IGNORE |,INTO|,"INSERT IGNORE INTO `cloud`.`configuration` VALUES ('Domain Defaults', 'DEFAULT', 'management-server', 'max.domain.user.vms','-1','The default maximum number of user VMs that can be deployed for a domain', '-1', NULL, NULL, 0);"
101,419802,39,Invalid expression / Unexpected token|,INSERT IGNORE |,INTO|,"INSERT IGNORE INTO `cloud`.`configuration` (`category`, `instance`, `component`, `name`, `value`, `default_value`, `description`) VALUES ('Advanced', 'DEFAULT', 'ManagementServer', 'ovm3.heartbeat.interval' , '10', '1', 'Interval value the checkheartbeat script uses before triggering the timeout for ovm3');"


### RESTRICT

In [226]:
no_into = no_install_thing[
    ~((no_install_thing.good2_file_parse_error_highlight.str.contains('INTO',case=False,regex=False)) &
    (
        (no_install_thing.original.str.contains('REM INSERTING',case=False,regex=False)) |
        (no_install_thing.original.str.contains('INSERT IGNORE INTO',case=False,regex=False))
    ))
]

restrict_chk = no_into[
    (no_into.good2_file_parse_error_highlight.str.contains('restrict',case=False,regex=False)) &
    (no_into.original.str.contains('ALTER TABLE',case=False,regex=False)) &
    (
        (no_into.original.str.contains('DELETE',case=False,regex=False)) |
        (no_into.original.str.contains('UPDATE',case=False,regex=False))
    )
]
print(get_stmt_and_file_count(restrict_chk))
#dot_check.sample(frac=0.6).head()  

[########################################] | 100% Completed | 26.96 s
[########################################] | 100% Completed | 29.62 s
(25255, 2612)


In [225]:
restrict_chk.sample(frac=0.6).head()  

[########################################] | 100% Completed | 809.30 ms


,file_id,statement_id,good2_parse_error,good2_file_parse_error_start_context,good2_file_parse_error_highlight,original
146,419828,31,Invalid key constraint|Invalid expression / Unexpected token|,d constraint fk_users_security_role_users_01 foreign key (users_id) references users (id) on delete |,restrict|,alter table users_security_role add constraint fk_users_security_role_users_01 foreign key (users_id) references users (id) on delete restrict on update restrict;
2791,420976,44,Invalid key constraint|Invalid expression / Unexpected token|,roles add constraint fk_users_roles_roles_02 foreign key (roles_id) references roles (id) on delete |,restrict|,alter table users_roles add constraint fk_users_roles_roles_02 foreign key (roles_id) references roles (id) on delete restrict on update restrict;
12268,424539,55,Invalid key constraint|Invalid expression / Unexpected token|,_�û����ջ���ַ foreign key (uid)\n references user_info (uid) on\n delete\n |,restrict|,alter table receive_address\n add constraint FK_�û����ջ���ַ foreign key (uid)\n references user_info (uid) on\n delete\n restrict on\n update restrict;
14864,425520,18,Invalid key constraint|Expecting )|Invalid expression / Unexpected token|,"IQUE(`warehouse_id`,`sku_id`),\n FOREIGN KEY (`sku_id`) REFERENCES `t_sku_product` (`id`) ON DELETE |",RESTRICT|,"CREATE TABLE `wms_inventory` (\n `id` bigint(20) NOT NULL AUTO_INCREMENT,\n `warehouse_id` bigint(20) NOT NULL COMMENT '仓库ID',\n `slot_id` bigint(20) DEFAULT NULL COMMENT '储位ID',\n `max_inventory` int(11) DEFAULT NULL COMMENT '库存上限',\n `min_inventory` int(11) DEFAULT '0' COMMENT '库存下限',\n `sku_id` bigint(20) NOT NULL COMMENT '库存量ID',\n `valid_sku` int(11) DEFAULT '0' COMMENT '可用库存量',\n `advance_quantities` int(11) DEFAULT '0' COMMENT '预购量',\n `transmit_quantities` int(11) DEFAULT '0' COMMENT '在途量',\n `order_count` int(11) NOT NULL DEFAULT '0' COMMENT '占用库存量',\n PRIMARY KEY (`id`),\n UNIQUE(`warehouse_id`,`sku_id`),\n FOREIGN KEY (`sku_id`) REFERENCES `t_sku_product` (`id`) ON DELETE RESTRICT,\n FOREIGN KEY (`warehouse_id`) REFERENCES `wms_warehouse` (`id`) ON DELETE RESTRICT\n) ENGINE=InnoDB DEFAULT CHARSET=utf8;"
3068,421089,34,Invalid key constraint|Invalid expression / Unexpected token|,FK_ASIGNATU_UN_NIVEL__NIVEL foreign key (IDNIVEL)\n references NIVEL (IDNIVEL)\n on delete |,restrict|,alter table ASIGNATURA\n add constraint FK_ASIGNATU_UN_NIVEL__NIVEL foreign key (IDNIVEL)\n references NIVEL (IDNIVEL)\n on delete restrict on update restrict;


### finalizing and writing down

In [227]:
no_restrict = no_into[
    ~((no_into.good2_file_parse_error_highlight.str.contains('restrict',case=False,regex=False)) &
    (no_into.original.str.contains('ALTER TABLE',case=False,regex=False)) &
    (
        (no_into.original.str.contains('DELETE',case=False,regex=False)) |
        (no_into.original.str.contains('UPDATE',case=False,regex=False))
    ))
]
print(get_stmt_and_file_count(no_restrict))
#dot_check.sample(frac=0.6).head()  

[########################################] | 100% Completed | 25.82 s
[########################################] | 100% Completed | 26.90 s
(1463548, 182387)


In [228]:
not_parsed_new_schema = pa.schema([])
not_parsed_new_schema = not_parsed_new_schema.append(pa.field("file_id",pa.string()))
not_parsed_new_schema = not_parsed_new_schema.append(pa.field("statement_id",pa.int64()))
not_parsed_new_schema = not_parsed_new_schema.append(pa.field("good2_parse_error",pa.string()))
not_parsed_new_schema = not_parsed_new_schema.append(pa.field("good2_file_parse_error_start_context",pa.string()))
not_parsed_new_schema = not_parsed_new_schema.append(pa.field("good2_file_parse_error_highlight",pa.string()))
not_parsed_new_schema = not_parsed_new_schema.append(pa.field("original",pa.string()))

In [229]:
#no_restrict.to_parquet('../out_new/sqlglot_unparsed_stmts_good_errors_hard_left_unparsed/',schema=not_parsed_new_schema,overwrite=True,
compute_kwargs={'scheduler':'processes'})

[########################################] | 100% Completed | 28.70 s


## Reading hard to parse stmts that are left

### Read

In [ ]:
not_parsed_new_schema = pa.schema([])
not_parsed_new_schema = not_parsed_new_schema.append(pa.field("file_id",pa.string()))
not_parsed_new_schema = not_parsed_new_schema.append(pa.field("statement_id",pa.int64()))
not_parsed_new_schema = not_parsed_new_schema.append(pa.field("good2_parse_error",pa.string()))
not_parsed_new_schema = not_parsed_new_schema.append(pa.field("good2_file_parse_error_start_context",pa.string()))
not_parsed_new_schema = not_parsed_new_schema.append(pa.field("good2_file_parse_error_highlight",pa.string()))
not_parsed_new_schema = not_parsed_new_schema.append(pa.field("original",pa.string()))

In [230]:
unparsed_hard = dd.read_parquet('../out_new/sqlglot_unparsed_stmts_good_errors_hard_left_unparsed/', columns = not_parsed_new_schema.names ,schema=not_parsed_new_schema, split_row_groups=True, calculate_divisions=True,engine='pyarrow')

In [231]:
print(get_stmt_and_file_count(unparsed_hard))

[########################################] | 100% Completed | 4.51 ss
[########################################] | 100% Completed | 3.83 ss
(1463548, 182387)


In [233]:
difficult_to_parse_file_ids = unparsed_hard['file_id'].unique().compute(scheduler='processes')

[########################################] | 100% Completed | 4.39 ss


In [234]:
difficult_to_parse_file_ids.to_csv('sqlglot_files_left_difficult_to_parse.csv',header=True,index=False)